In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, average_precision_score, confusion_matrix, classification_report
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# =============================================================================
# Project: Next-year financial distress prediction (firm-year panel)
# Data Science Lifecycle focus in this cell:
#   - Setup & configuration (temporal split conventions, leakage control parameters)
#   - Panel integrity (types, deduplication, stable firm identifier)
#   - Define label_year used consistently for splitting and target alignment
# =============================================================================

# ----------------------------
# Configuration (split & preprocessing parameters)
# ----------------------------
FILE_NAME = "data.csv"

TRAIN_CUTOFF_LABEL_YEAR = 2022   # label_year <= cutoff → train/val pool; after cutoff → test
VAL_YEARS = 1                    # last N years within the pool are validation
N_SPLITS_TIME_CV = 4             # rolling time-based folds for sanity checks

WINSOR_LOWER_Q = 0.01            # winsorization lower quantile (train-only)
WINSOR_UPPER_Q = 0.99            # winsorization upper quantile (train-only)

REQUIRED_KEYS = ["gvkey", "fyear"]
TARGET_COL = "target_next_year_distress"


# ----------------------------
# Utilities (robust numeric ops for ratios)
# ----------------------------

def to_float_numpy(x) -> np.ndarray:
    """Convert series/array-like to float numpy array, coercing non-numeric to NaN."""
    s = pd.to_numeric(x, errors="coerce")
    return s.to_numpy(dtype=float) if hasattr(s, "to_numpy") else np.asarray(s, dtype=float)
def safe_divide(a, b) -> np.ndarray:
    """Elementwise divide a/b with NaN when division is invalid (0 or non-finite)."""
    a = to_float_numpy(a)
    b = to_float_numpy(b)
    out = np.full_like(a, np.nan, dtype=float)
    np.divide(a, b, out=out, where=(b != 0) & np.isfinite(a) & np.isfinite(b))
    return out

def rolling_year_folds(
    df_in: pd.DataFrame, year_col: str = "label_year", n_splits: int = 4, min_train_years: int = 3
) -> list[tuple[np.ndarray, np.ndarray, np.ndarray, int]]:
    """
    Create expanding-window time folds:
      train years: first (min_train_years + k) years
      val year:    next year
    Returns: list of (train_idx, val_idx, train_years, val_year)
    """
    years_sorted = np.sort(df_in[year_col].dropna().unique())
    if len(years_sorted) <= min_train_years:
        return []
    n_splits = min(n_splits, len(years_sorted) - min_train_years)

    folds_out = []
    for k in range(n_splits):
        train_years = years_sorted[: min_train_years + k]
        val_year = int(years_sorted[min_train_years + k])

        train_idx = df_in.index[df_in[year_col].isin(train_years)].to_numpy()
        val_idx = df_in.index[df_in[year_col] == val_year].to_numpy()
        folds_out.append((train_idx, val_idx, train_years, val_year))

    return folds_out


# =============================================================================
# 1) Data acquisition & panel hygiene
# =============================================================================
df = pd.read_csv(FILE_NAME, low_memory=False)

# Convert datadate if present
if "datadate" in df.columns:
    df["datadate"] = pd.to_datetime(df["datadate"], errors="coerce")

# Create stable firm id + de-duplicate firm-year (keep last record)
df["firm_id"] = df["gvkey"]
df = (
    df.sort_values(["firm_id", "fyear"])
      .drop_duplicates(subset=["firm_id", "fyear"], keep="last")
      .reset_index(drop=True)
)

# Label year: predict distress in the next fiscal year
df["label_year"] = df["fyear"] + 1

# =============================================================================
# 2) Split scaffolding (define train/val pool years via label_year)
# =============================================================================
pool_mask = df["label_year"] <= TRAIN_CUTOFF_LABEL_YEAR
pool_years = np.sort(df.loc[pool_mask, "label_year"].dropna().unique())
val_years = pool_years[-VAL_YEARS:] if len(pool_years) else np.array([], dtype=int)

# This mask is ONLY used for imputations (train-only information)
train_mask_for_imputation = pool_mask & (~df["label_year"].isin(val_years))


In [ ]:
# =============================================================================
# Data Cleaning & Missing-Data Handling (leakage-aware)
# Purpose:
#   - Quantify missingness and distribution properties before intervention
#   - Preserve informative missingness via miss_* indicators
#   - Impute financial statement inputs using TRAIN-only information:
#       (1) within-firm past values (lag-1) where economically sensible
#       (2) peer medians by year×size_decile on size-scaled ratios (ratio space)
#       (3) KNN imputation (TRAIN-fit) for selected core balance-sheet items
#   - Re-run EDA after imputation to audit how strongly imputations alter the data
# =============================================================================

import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

RAW_INPUTS_FOR_FE = [
    "aco","act","ao","aoloch","ap","apalch","aqc","at","caps","capx","ceq","che","chech","csho","cstk","cstke",
    "datadate","dlc","dlcch","dltis","dltr","dltt","do","dp","dpc","dv","dvc","dvp","dvt","esubc","exre",
    "fiao","fincf","fopo","fyear","gvkey","ib","ibadj","ibc","intan","invch","invt","ismod","ivaco","ivaeq",
    "ivao","ivch","ivncf","ivstch","lco","lct","lt","mibt","mkvalt","niadj","nopi","oancf","oibdp","ppent",
    "prcc_c","prcc_f","prstkc","pstk","pstkn","pstkr","re","recch","rect","seq","siv","spi","sppe","sppiv",
    "sstk","tstk","txach","txbcof","txdc","txditc","txp","txt","xi","xido","xidoc","xint",
    # optional identifiers present in many extracts:
    "conm","consol","datafmt","indfmt",
]
raw = [c for c in RAW_INPUTS_FOR_FE if c in df.columns]

# ---------------------------------------------------------------------------
# 3.0 Ensure keys exist + types
# ---------------------------------------------------------------------------
if "firm_id" not in df.columns:
    if "gvkey" in df.columns:
        df["firm_id"] = df["gvkey"]
    else:
        raise ValueError("Need either firm_id or gvkey in df to run panel imputations.")

if "fyear" in df.columns:
    df["fyear"] = pd.to_numeric(df["fyear"], errors="coerce")

# ---------------------------------------------------------------------------
# 3.1 Drop rows with missing critical identifiers (do not impute these)
# ---------------------------------------------------------------------------
NON_IMPUTE_DROP = [c for c in ["gvkey", "datadate", "fyear", "conm", "datafmt", "indfmt", "consol"] if c in df.columns]
if NON_IMPUTE_DROP:
    before_n = df.shape[0]
    df = df.dropna(subset=NON_IMPUTE_DROP).copy()
    after_n = df.shape[0]
    if after_n < before_n:
        print(f"[INFO] Dropped {before_n - after_n:,} rows due to missing non-imputable ID/meta fields: {NON_IMPUTE_DROP}")

# Ensure train mask aligns after drops
if isinstance(train_mask_for_imputation, pd.Series):
    train_mask_for_imputation = train_mask_for_imputation.reindex(df.index).fillna(False).astype(bool)

# Rebuild raw after potential drop
raw = [c for c in RAW_INPUTS_FOR_FE if c in df.columns]

# ---------------------------------------------------------------------------
# 3.2 EDA BEFORE imputation (missingness + distribution snapshot)
# ---------------------------------------------------------------------------
df_raw_pre = df[raw].copy(deep=True)

pre_miss = pd.DataFrame(
    {
        "col": raw,
        "n": [int(df_raw_pre[c].shape[0]) for c in raw],
        "n_na_pre": [int(df_raw_pre[c].isna().sum()) for c in raw],
        "pct_na_pre": [float(df_raw_pre[c].isna().mean() * 100.0) for c in raw],
        "train_n": [int(train_mask_for_imputation.sum()) for _ in raw],
        "train_pct_na_pre": [
            float(df_raw_pre.loc[train_mask_for_imputation, c].isna().mean() * 100.0) for c in raw
        ],
    }
).sort_values("pct_na_pre", ascending=False)

print("\n=== EDA (BEFORE imputation): Missingness on raw inputs ===")
print(pre_miss.round(4).head(50))

# Numeric distribution summary (coerce non-numeric to NaN)
if raw:
    x_pre = df_raw_pre[raw].apply(pd.to_numeric, errors="coerce").replace([np.inf, -np.inf], np.nan)
    q_pre = x_pre.quantile([0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).T
    pre_dist = pd.DataFrame(
        {
            "n_nonmiss_pre": x_pre.notna().sum(),
            "mean_pre": x_pre.mean(),
            "std_pre": x_pre.std(ddof=0),
            "min_pre": x_pre.min(),
            "p01_pre": q_pre[0.01],
            "p05_pre": q_pre[0.05],
            "p25_pre": q_pre[0.25],
            "p50_pre": q_pre[0.50],
            "p75_pre": q_pre[0.75],
            "p95_pre": q_pre[0.95],
            "p99_pre": q_pre[0.99],
            "max_pre": x_pre.max(),
        }
    )
    print("\n=== EDA (BEFORE imputation): Distribution summary (raw inputs) ===")
    print(pre_dist.round(4).sort_values("n_nonmiss_pre", ascending=True).head(50))

# ---------------------------------------------------------------------------
# 3.3 Missingness flags (ALWAYS create before imputations)
# ---------------------------------------------------------------------------
for c in raw:
    df[f"miss_{c}"] = df[c].isna().astype("int8")

# Helper: originally-observed indicator
def _obs(col: str) -> pd.Series:
    m = f"miss_{col}"
    if m in df.columns:
        return (df[m] == 0)
    return df[col].notna()

# ---------------------------------------------------------------------------
# 3.4 Grouping key for peer-based imputations (no industry used)
#   - Use TRAIN-derived size_decile to avoid mixing microcaps with mega-caps
# ---------------------------------------------------------------------------
df = df.sort_values(["firm_id", "fyear"]).copy()

# TRAIN-derived size deciles on log(at) using observed TRAIN values only
if "at" in df.columns:
    at_train = pd.to_numeric(df.loc[train_mask_for_imputation, "at"], errors="coerce")
    log_at_train = np.log(at_train.where(at_train > 0))
    qs = np.linspace(0, 1, 11)
    edges = np.unique(np.nanquantile(log_at_train.dropna(), qs))
    if edges.size >= 3:
        log_at_all = np.log(pd.to_numeric(df["at"], errors="coerce").where(pd.to_numeric(df["at"], errors="coerce") > 0))
        df["size_decile"] = pd.cut(log_at_all, bins=edges, include_lowest=True, labels=False)
    else:
        df["size_decile"] = np.nan
else:
    df["size_decile"] = np.nan

group_cols = ["fyear", "size_decile"]

# ---------------------------------------------------------------------------
# 3.5 Step 1: Construct / reconcile FIRST (no leakage: contemporaneous or lag only)
# ---------------------------------------------------------------------------
# 3.5.1 Mild lag-1 fill for prices, then mkvalt = prcc_f * csho if missing
for px in ["prcc_f", "prcc_c"]:
    if px in df.columns:
        df[px] = df[px].where(df[px].notna(), df.groupby("firm_id")[px].shift(1))

if all(c in df.columns for c in ["mkvalt", "prcc_f", "csho"]):
    mkvalt_miss = df["mkvalt"].isna()
    mkvalt_calc = pd.to_numeric(df["prcc_f"], errors="coerce") * pd.to_numeric(df["csho"], errors="coerce")
    df.loc[mkvalt_miss & mkvalt_calc.notna(), "mkvalt"] = mkvalt_calc.loc[mkvalt_miss & mkvalt_calc.notna()]

# 3.5.2 Reconstruct change variables from level differences (fill only if change var is missing)
def _fill_change_from_levels(change_col, level_col):
    if change_col in df.columns and level_col in df.columns:
        miss = df[change_col].isna()
        lvl = pd.to_numeric(df[level_col], errors="coerce")
        lag_lvl = pd.to_numeric(df.groupby("firm_id")[level_col].shift(1), errors="coerce")
        recon = lvl - lag_lvl
        df.loc[miss & recon.notna(), change_col] = recon.loc[miss & recon.notna()]

_fill_change_from_levels("dlcch", "dlc")
_fill_change_from_levels("recch", "rect")
_fill_change_from_levels("invch", "invt")
_fill_change_from_levels("chech", "che")

# ---------------------------------------------------------------------------
# 3.6 Sparse zero-fill for structurally zero-inflated items (TRAIN-validated)
#   - Keeps miss_* as informative signal
# ---------------------------------------------------------------------------
SPARSE_CANDIDATES = [c for c in [
    "txach","txdc","txbcof","spi","siv","sppiv","ivstch","sppe","esubc","dvp","dv","dvc","dvt"
] if c in df.columns]

def _zero_fill_if_sparse(col: str, zero_share_thresh: float = 0.70, min_obs: int = 1000) -> bool:
    mflag = f"miss_{col}"
    if mflag not in df.columns:
        return False
    obs_mask = train_mask_for_imputation & (df[mflag] == 0)
    s = pd.to_numeric(df.loc[obs_mask, col], errors="coerce").replace([np.inf, -np.inf], np.nan)
    if s.notna().sum() < min_obs:
        return False
    zero_share = float((s == 0).mean())
    med = float(s.median()) if s.notna().any() else np.nan
    if np.isfinite(med) and (med == 0.0) and (zero_share >= zero_share_thresh):
        df.loc[df[col].isna(), col] = 0.0
        return True
    return False

for c in SPARSE_CANDIDATES:
    _zero_fill_if_sparse(c)

# Explicit “tax components -> 0” rule (FFO proxy safety)
for c in ["txt", "txdc", "txach"]:
    if c in df.columns:
        df.loc[df[c].isna(), c] = 0.0

# ---------------------------------------------------------------------------
# 3.7 Step 3: Stocks — lag-1 fill -> peer median of ratio (x/at) with ratio-space clipping
# ---------------------------------------------------------------------------
STOCKS = [c for c in [
    "aco","act","ao","ap","at","caps","ceq","che","csho","cstk","dlc","dltt","intan","invt","lco","lct","lt",
    "mibt","ppent","pstk","pstkn","pstkr","re","rect","seq","tstk","ivaeq","mkvalt"
] if c in df.columns]

# lag-1 fill (past-only)
if STOCKS:
    lag1 = df.groupby("firm_id")[STOCKS].shift(1)
    df[STOCKS] = df[STOCKS].where(df[STOCKS].notna(), lag1)

# non-negativity only where economically necessary
NONNEG_STOCKS = set([c for c in STOCKS if c in {
    "aco","act","ao","ap","at","caps","che","csho","cstk","dlc","dltt","intan","invt","lco","lct","lt",
    "mibt","mkvalt","ppent","pstk","pstkn","pstkr","rect","tstk","ivaeq"
}])

def _fit_ratio_stats(train_df: pd.DataFrame, col: str, base: str = "at", qlo=0.01, qhi=0.99):
    mcol = f"miss_{col}"
    mbase = f"miss_{base}"
    s = pd.to_numeric(train_df[col], errors="coerce")
    b = pd.to_numeric(train_df[base], errors="coerce") if base in train_df.columns else None

    obs_s = (train_df[mcol] == 0) if mcol in train_df.columns else s.notna()

    if b is None:
        tr = train_df.loc[obs_s, group_cols + [col]].copy()
        tr[col] = pd.to_numeric(tr[col], errors="coerce").replace([np.inf, -np.inf], np.nan)
        grp = tr.groupby(group_cols)[col].median()
        overall = float(tr[col].median()) if tr[col].notna().any() else 0.0
        return ("level", overall, grp, np.nan, np.nan)

    obs_b = (train_df[mbase] == 0) if mbase in train_df.columns else b.notna()
    valid = obs_s & obs_b & s.notna() & b.notna() & (b > 0)

    if valid.sum() < 200:
        tr = train_df.loc[obs_s, group_cols + [col]].copy()
        tr[col] = pd.to_numeric(tr[col], errors="coerce").replace([np.inf, -np.inf], np.nan)
        grp = tr.groupby(group_cols)[col].median()
        overall = float(tr[col].median()) if tr[col].notna().any() else 0.0
        return ("level", overall, grp, np.nan, np.nan)

    ratio = (s[valid] / b[valid]).replace([np.inf, -np.inf], np.nan).dropna()
    overall = float(ratio.median()) if ratio.notna().any() else 0.0
    r_lo = float(ratio.quantile(qlo))
    r_hi = float(ratio.quantile(qhi))

    tmp = train_df.loc[valid, group_cols].copy()
    tmp["_ratio_"] = ratio.values
    grp = tmp.groupby(group_cols)["_ratio_"].median()
    return ("ratio", overall, grp, r_lo, r_hi)

def _apply_ratio_stats(df_all: pd.DataFrame, col: str, fit_obj, base: str = "at", nonneg: bool = False):
    kind, overall, grp, r_lo, r_hi = fit_obj
    miss = df_all[col].isna()
    if not miss.any():
        return

    if kind == "ratio" and base in df_all.columns:
        b = pd.to_numeric(df_all.loc[miss, base], errors="coerce")
        g = df_all.loc[miss, group_cols]

        if len(group_cols) == 1:
            mapped = g[group_cols[0]].map(grp)
        else:
            keys = list(map(tuple, g[group_cols].to_numpy()))
            mapped = pd.Series(keys, index=g.index).map(grp)

        r = pd.to_numeric(mapped, errors="coerce").replace([np.inf, -np.inf], np.nan).fillna(overall)

        # clip in ratio space (TRAIN-observed band)
        if np.isfinite(r_lo) and np.isfinite(r_hi) and (r_lo < r_hi):
            r = r.clip(r_lo, r_hi)

        fill = r * b
        fill = fill.where(b.notna() & (b > 0), np.nan)
        df_all.loc[miss & fill.notna(), col] = fill.loc[miss & fill.notna()].to_numpy()

    # fallback: group level medians (TRAIN-fit, observed-only)
    miss2 = df_all[col].isna()
    if miss2.any():
        mflag = f"miss_{col}"
        obs_train_mask = train_mask_for_imputation
        if mflag in df_all.columns:
            obs_train_mask = obs_train_mask & (df_all[mflag] == 0)

        tr = df_all.loc[obs_train_mask, group_cols + [col]].copy()
        tr[col] = pd.to_numeric(tr[col], errors="coerce").replace([np.inf, -np.inf], np.nan)

        lvl_overall = float(tr[col].median()) if tr[col].notna().any() else 0.0
        lvl_grp = tr.groupby(group_cols)[col].median()

        g2 = df_all.loc[miss2, group_cols]
        if len(group_cols) == 1:
            mapped2 = g2[group_cols[0]].map(lvl_grp)
        else:
            keys2 = list(map(tuple, g2[group_cols].to_numpy()))
            mapped2 = pd.Series(keys2, index=g2.index).map(lvl_grp)

        fill2 = pd.to_numeric(mapped2, errors="coerce").replace([np.inf, -np.inf], np.nan).fillna(lvl_overall)
        if nonneg:
            fill2 = fill2.clip(lower=0.0)
        df_all.loc[miss2, col] = fill2.to_numpy()

# Fit on training
tr_all = df.loc[train_mask_for_imputation].copy()
stock_fits = {c: _fit_ratio_stats(tr_all, c, base="at") for c in STOCKS}

# Apply to full df
for c in STOCKS:
    _apply_ratio_stats(df, c, stock_fits[c], base="at", nonneg=(c in NONNEG_STOCKS))

# ---------------------------------------------------------------------------
# 3.8 Step 4: Flows / income variables — ratio-median imputation (leakage-aware)
# ---------------------------------------------------------------------------
FLOWS = [c for c in ["ib","ibadj","ibc","niadj","nopi","oibdp","dp","oancf","fincf","ivncf","xint"] if c in df.columns]

# Debt-aware base for xint
if "xint" in df.columns and all(c in df.columns for c in ["dlc", "dltt"]):
    df["_td_for_xint"] = (
        pd.to_numeric(df["dlc"], errors="coerce").fillna(0.0)
        + pd.to_numeric(df["dltt"], errors="coerce").fillna(0.0)
    )
else:
    df["_td_for_xint"] = np.nan

# If total debt <= 0 and xint missing -> 0
if "xint" in df.columns and "_td_for_xint" in df.columns:
    td = pd.to_numeric(df["_td_for_xint"], errors="coerce").fillna(0.0)
    df.loc[df["xint"].isna() & (td <= 0), "xint"] = 0.0

flow_fits = {}
for c in FLOWS:
    base = "_td_for_xint" if (c == "xint" and "_td_for_xint" in df.columns) else "at"
    flow_fits[c] = _fit_ratio_stats(tr_all, c, base=base)

for c in FLOWS:
    base = "_td_for_xint" if (c == "xint" and "_td_for_xint" in df.columns) else "at"
    _apply_ratio_stats(df, c, flow_fits[c], base=base, nonneg=False)

# ---------------------------------------------------------------------------
# 3.9 KNN imputation (TRAIN-fit) for selected core balance-sheet items
#   - Applies ONLY to originally-missing rows (miss_* == 1)
#   - Uses a robust signed-log transform for magnitudes
# ---------------------------------------------------------------------------
KNN_TARGETS = [c for c in [
    "at","act","lct","lt","seq","dlc","dltt","che","ppent","rect","invt","re","ceq","caps"
] if c in df.columns]

# Non-negativity for clearly non-negative magnitudes (do NOT force seq/ceq/re to be non-negative)
KNN_NONNEG = set([c for c in KNN_TARGETS if c in {"at","act","lct","lt","dlc","dltt","che","ppent","rect","invt","caps"}])

if KNN_TARGETS:
    # KNN feature set (kept compact and available in your raw inputs)
    knn_feats = []
    for c in ["fyear", "size_decile", "mkvalt", "csho", "prcc_f", "prcc_c"]:
        if c in df.columns:
            knn_feats.append(c)
    knn_feats = list(dict.fromkeys(knn_feats))  # preserve order, unique

    knn_cols = list(dict.fromkeys(knn_feats + KNN_TARGETS))
    X = df[knn_cols].apply(pd.to_numeric, errors="coerce").replace([np.inf, -np.inf], np.nan)

    # For KNN, re-instate NaN only for the targets that were originally missing
    for t in KNN_TARGETS:
        mflag = f"miss_{t}"
        if mflag in df.columns:
            X.loc[df[mflag] == 1, t] = np.nan

    # Fill size_decile NaN with TRAIN median decile for distance stability
    if "size_decile" in X.columns:
        sd_train = X.loc[train_mask_for_imputation, "size_decile"]
        sd_fill = float(sd_train.median()) if sd_train.notna().any() else 5.0
        X["size_decile"] = X["size_decile"].fillna(sd_fill)

    # Robust transform: signed log1p for magnitudes; keep fyear/size_decile in levels
    def _signed_log1p(a: pd.Series) -> pd.Series:
        v = pd.to_numeric(a, errors="coerce")
        return np.sign(v) * np.log1p(np.abs(v))

    Z = X.copy()
    for c in Z.columns:
        if c not in {"fyear", "size_decile"}:
            Z[c] = _signed_log1p(Z[c])

    # Standardize using TRAIN-only stats (nan-safe)
    mu = Z.loc[train_mask_for_imputation].mean(skipna=True)
    sd = Z.loc[train_mask_for_imputation].std(skipna=True, ddof=0).replace(0.0, 1.0)
    sd = sd.fillna(1.0)
    mu = mu.fillna(0.0)

    Zs = (Z - mu) / sd

    # Fit KNN on TRAIN only; transform all rows
    imputer = KNNImputer(n_neighbors=25, weights="distance", metric="nan_euclidean")
    imputer.fit(Zs.loc[train_mask_for_imputation].to_numpy(dtype=float))
    Zs_imp = imputer.transform(Zs.to_numpy(dtype=float))
    Zs_imp = pd.DataFrame(Zs_imp, index=Zs.index, columns=Zs.columns)

    # Unscale
    Z_imp = Zs_imp * sd + mu

    # Invert signed log for magnitudes
    X_imp = X.copy()
    for c in Z_imp.columns:
        if c in {"fyear", "size_decile"}:
            X_imp[c] = Z_imp[c]
        else:
            v = pd.to_numeric(Z_imp[c], errors="coerce")
            X_imp[c] = np.sign(v) * (np.expm1(np.abs(v)))

    # Write back ONLY for originally-missing targets
    for t in KNN_TARGETS:
        mflag = f"miss_{t}"
        if mflag in df.columns:
            miss_mask = (df[mflag] == 1)
            df.loc[miss_mask, t] = pd.to_numeric(X_imp.loc[miss_mask, t], errors="coerce").to_numpy()

            if t in KNN_NONNEG:
                df.loc[miss_mask, t] = pd.to_numeric(df.loc[miss_mask, t], errors="coerce").clip(lower=0.0)

# ---------------------------------------------------------------------------
# 3.10 Change variables AFTER level imputations:
#   - Fill only if change is missing
#   - Reconstruct only when both levels were originally observed; otherwise impute change directly
# ---------------------------------------------------------------------------
def _fill_change_after_levels(change_col, level_col):
    if change_col in df.columns and level_col in df.columns:
        lvl = pd.to_numeric(df[level_col], errors="coerce")
        lag_lvl = pd.to_numeric(df.groupby("firm_id")[level_col].shift(1), errors="coerce")
        recon = lvl - lag_lvl

        miss_now = df[change_col].isna()

        # only reconstruct when both levels were originally observed
        obs_now = _obs(level_col)
        obs_lag = obs_now.groupby(df["firm_id"]).shift(1).fillna(False)
        can_recon = miss_now & obs_now & obs_lag & recon.notna()
        df.loc[can_recon, change_col] = recon.loc[can_recon]

        # remaining missing: direct group-median impute of change (TRAIN-fit, observed-only if possible)
        miss2 = df[change_col].isna()
        if miss2.any():
            mflag = f"miss_{change_col}"
            obs_train = train_mask_for_imputation
            if mflag in df.columns:
                obs_train = obs_train & (df[mflag] == 0)

            tr = df.loc[obs_train, group_cols + [change_col]].copy()
            tr[change_col] = pd.to_numeric(tr[change_col], errors="coerce").replace([np.inf, -np.inf], np.nan)
            overall = float(tr[change_col].median()) if tr[change_col].notna().any() else 0.0
            grp = tr.groupby(group_cols)[change_col].median()

            g = df.loc[miss2, group_cols]
            if len(group_cols) == 1:
                mapped = g[group_cols[0]].map(grp)
            else:
                keys = list(map(tuple, g[group_cols].to_numpy()))
                mapped = pd.Series(keys, index=g.index).map(grp)

            fill = pd.to_numeric(mapped, errors="coerce").replace([np.inf, -np.inf], np.nan).fillna(overall)
            df.loc[miss2, change_col] = fill.to_numpy()

_fill_change_after_levels("dlcch", "dlc")
_fill_change_after_levels("recch", "rect")
_fill_change_after_levels("invch", "invt")
_fill_change_after_levels("chech", "che")
if "apalch" in df.columns and "ap" in df.columns:
    _fill_change_after_levels("apalch", "ap")

# ---------------------------------------------------------------------------
# 3.11 Guardrail: cap ONLY-imputed values (miss_* == 1) to TRAIN-observed quantile band
# ---------------------------------------------------------------------------
REG_NONNEG = set(["at","act","lct","lt","dlc","dltt","che","ppent","rect","invt","caps"])  # for this script

def _cap_imputed_to_train_quantiles(
    df_all: pd.DataFrame,
    col: str,
    lower_q: float = 0.01,
    upper_q: float = 0.99,
    nonneg: bool = False,
) -> None:
    flag = f"miss_{col}"
    if col not in df_all.columns or flag not in df_all.columns:
        return

    miss_mask = df_all[flag].astype(bool)
    if not miss_mask.any():
        return

    # bounds from TRAIN & originally-observed values
    obs = pd.to_numeric(df_all.loc[train_mask_for_imputation & (~miss_mask), col], errors="coerce").replace(
        [np.inf, -np.inf], np.nan
    )
    if obs.notna().sum() < 200:
        obs = pd.to_numeric(df_all.loc[train_mask_for_imputation, col], errors="coerce").replace([np.inf, -np.inf], np.nan)

    if obs.notna().sum() == 0:
        return

    lo = float(obs.quantile(lower_q))
    hi = float(obs.quantile(upper_q))

    if nonneg:
        lo = max(0.0, lo) if np.isfinite(lo) else 0.0

    s = pd.to_numeric(df_all.loc[miss_mask, col], errors="coerce").replace([np.inf, -np.inf], np.nan)
    s = s.clip(lo, hi)
    if nonneg:
        s = s.clip(lower=0.0)
    df_all.loc[miss_mask, col] = s.to_numpy()

CAP_COLS = sorted(
    (
        set(STOCKS + FLOWS + ["dlcch", "apalch", "recch", "invch", "chech"])
        | set(KNN_TARGETS)
    ) & set(df.columns)
)

for c in CAP_COLS:
    _cap_imputed_to_train_quantiles(
        df,
        c,
        lower_q=0.01,
        upper_q=0.99,
        nonneg=(c in NONNEG_STOCKS) or (c in REG_NONNEG),
    )

# ---------------------------------------------------------------------------
# 3.12 EDA AFTER imputation (missingness reduction + distribution deltas)
# ---------------------------------------------------------------------------
df_raw_post = df[raw].copy(deep=True)

post_miss = pd.DataFrame(
    {
        "col": raw,
        "n_na_post": [int(df_raw_post[c].isna().sum()) for c in raw],
        "pct_na_post": [float(df_raw_post[c].isna().mean() * 100.0) for c in raw],
        "train_pct_na_post": [
            float(df_raw_post.loc[train_mask_for_imputation, c].isna().mean() * 100.0) for c in raw
        ],
    }
)

changes = pre_miss.merge(post_miss, on="col", how="left")
changes["n_imputed"] = changes["n_na_pre"] - changes["n_na_post"]
changes["pct_points_na_reduction"] = changes["pct_na_pre"] - changes["pct_na_post"]

x_post = df_raw_post[raw].apply(pd.to_numeric, errors="coerce").replace([np.inf, -np.inf], np.nan)
q_post = x_post.quantile([0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).T
post_dist = pd.DataFrame(
    {
        "n_nonmiss_post": x_post.notna().sum(),
        "mean_post": x_post.mean(),
        "std_post": x_post.std(ddof=0),
        "min_post": x_post.min(),
        "p01_post": q_post[0.01],
        "p05_post": q_post[0.05],
        "p25_post": q_post[0.25],
        "p50_post": q_post[0.50],
        "p75_post": q_post[0.75],
        "p95_post": q_post[0.95],
        "p99_post": q_post[0.99],
        "max_post": x_post.max(),
    }
)

pre_dist_key = pre_dist[["n_nonmiss_pre","mean_pre","std_pre","p01_pre","p50_pre","p99_pre"]].copy() if raw else pd.DataFrame()
post_dist_key = post_dist[["n_nonmiss_post","mean_post","std_post","p01_post","p50_post","p99_post"]].copy() if raw else pd.DataFrame()

dist_delta = pre_dist_key.join(post_dist_key, how="outer")
dist_delta["delta_mean"] = dist_delta["mean_post"] - dist_delta["mean_pre"]
dist_delta["delta_std"]  = dist_delta["std_post"]  - dist_delta["std_pre"]
dist_delta["delta_p50"]  = dist_delta["p50_post"]  - dist_delta["p50_pre"]

# Imputed-only diagnostics (based on original missingness in df_raw_pre)
rows = []
for c in raw:
    pre_na_mask = df_raw_pre[c].isna()
    n_imp = int(pre_na_mask.sum())
    if n_imp == 0:
        rows.append((c, 0, np.nan, np.nan, np.nan, np.nan, np.nan))
        continue

    imp_vals = pd.to_numeric(df_raw_post.loc[pre_na_mask, c], errors="coerce").replace([np.inf, -np.inf], np.nan)
    obs_vals = pd.to_numeric(df_raw_pre.loc[~pre_na_mask, c], errors="coerce").replace([np.inf, -np.inf], np.nan)

    rows.append(
        (
            c,
            n_imp,
            float(imp_vals.mean()) if imp_vals.notna().any() else np.nan,
            float(imp_vals.median()) if imp_vals.notna().any() else np.nan,
            float(imp_vals.std(ddof=0)) if imp_vals.notna().any() else np.nan,
            float(obs_vals.mean()) if obs_vals.notna().any() else np.nan,
            float(obs_vals.median()) if obs_vals.notna().any() else np.nan,
        )
    )

imputed_only = pd.DataFrame(
    rows,
    columns=["col","n_imputed","imputed_mean","imputed_median","imputed_std","observed_mean_pre","observed_median_pre"],
).set_index("col")

print("\n=== EDA (AFTER imputation): Missingness on raw inputs + change ===")
cols_show = [
    "col", "n", "n_na_pre", "pct_na_pre", "n_na_post", "pct_na_post",
    "n_imputed", "pct_points_na_reduction", "train_pct_na_pre", "train_pct_na_post",
]
print(
    changes[cols_show]
    .sort_values(["n_imputed","pct_points_na_reduction"], ascending=[False, False])
    .round(4)
    .head(50)
)

print("\n=== Change analysis: Distribution deltas (post - pre) on raw inputs ===")
print(
    dist_delta[["n_nonmiss_pre","n_nonmiss_post","delta_mean","delta_std","delta_p50"]]
    .sort_values("delta_mean", key=lambda s: s.abs(), ascending=False)
    .round(6)
    .head(50)
)

print("\n=== Change analysis: Imputed-only vs observed (pre) summary ===")
print(
    imputed_only.assign(
        mean_gap_imputed_minus_observed=lambda d: d["imputed_mean"] - d["observed_mean_pre"],
        median_gap_imputed_minus_observed=lambda d: d["imputed_median"] - d["observed_median_pre"],
    )
    .sort_values("n_imputed", ascending=False)
    .round(6)
    .head(50)
)



=== EDA (BEFORE imputation): Missingness on raw inputs ===
       col      n  n_na_pre  pct_na_pre  train_n  train_pct_na_pre
18   dlcch  75005     33143     44.1877    48458           42.9630
5   apalch  75005     30371     40.4920    48458           39.0214
75   txach  75005     22791     30.3860    48458           29.2501
48  ivstch  75005     19194     25.5903    48458           23.0282
66   recch  75005     12589     16.7842    48458           16.5938
53  mkvalt  75005     12350     16.4656    48458           17.0189
71    sppe  75005     12239     16.3176    48458           16.4307
1      act  75005     10721     14.2937    48458           14.6581
50     lct  75005     10695     14.2590    48458           14.5982
84    xint  75005     10536     14.0471    48458           13.9234
78  txditc  75005      9069     12.0912    48458           12.2044
79     txp  75005      7942     10.5886    48458           10.6917
29   esubc  75005      6798      9.0634    48458            8.8964
49

In [ ]:
# =============================================================================
# Feature Engineering & Target Construction
#   - Build leverage / coverage / cash-flow-to-debt ratios commonly used in credit analysis
#   - Define a 'highly leveraged' distress proxy from multiple ratio-based conditions
#   - Create the supervised-learning target: next-year distress within the same firm
# =============================================================================

firm_col = "firm_id"

dlc = pd.to_numeric(df.get("dlc", np.nan), errors="coerce")
dltt = pd.to_numeric(df.get("dltt", np.nan), errors="coerce")
df["total_debt"] = pd.concat([dlc, dltt], axis=1).sum(axis=1, min_count=1)

seq = pd.to_numeric(df.get("seq", np.nan), errors="coerce")
mibt = pd.to_numeric(df.get("mibt", 0.0), errors="coerce")
df["equity_plus_mi_sp"] = seq + mibt
df["total_capital_sp"] = df["total_debt"] + df["equity_plus_mi_sp"]

# --- CHANGED (minimal): handle total capital <= 0 as extreme leverage + flag ---
cap_s = pd.to_numeric(df["total_capital_sp"], errors="coerce")
df["cap_nonpos_flag"] = (cap_s.notna() & (cap_s <= 0)).astype("int8")
df["sp_debt_to_capital"] = safe_divide(df["total_debt"], cap_s)
df.loc[df["cap_nonpos_flag"] == 1, "sp_debt_to_capital"] = np.inf
# ---------------------------------------------------------------------------

oibdp = pd.to_numeric(df.get("oibdp", np.nan), errors="coerce")
xint = pd.to_numeric(df.get("xint", np.nan), errors="coerce")

# --- CHANGED (minimal): handle EBITDA <= 0 as extreme leverage + flag ---
df["ebitda_nonpos_flag"] = (oibdp.notna() & (oibdp <= 0)).astype("int8")
df["sp_debt_to_ebitda"] = safe_divide(df["total_debt"], oibdp)
df.loc[df["ebitda_nonpos_flag"] == 1, "sp_debt_to_ebitda"] = np.inf
# ---------------------------------------------------------------------------

# --- OPTIONAL transparency (simple): flag if tax proxy components missing (no complex proxying) ---
txt_raw = pd.to_numeric(df.get("txt", np.nan), errors="coerce")
txdc_raw = pd.to_numeric(df.get("txdc", np.nan), errors="coerce")
txach_raw = pd.to_numeric(df.get("txach", np.nan), errors="coerce")
df["tax_proxy_incomplete"] = (txt_raw.isna() | txdc_raw.isna() | txach_raw.isna()).astype("int8")

txt = txt_raw.fillna(0.0)
txdc = txdc_raw.fillna(0.0)
txach = txach_raw.fillna(0.0)
df["cash_tax_paid_proxy"] = txt - txdc - txach

df["ffo_proxy"] = oibdp - xint - pd.to_numeric(df["cash_tax_paid_proxy"], errors="coerce")
df["sp_ffo_to_debt"] = safe_divide(df["ffo_proxy"], df["total_debt"])

oancf = pd.to_numeric(df.get("oancf", np.nan), errors="coerce")
capx = pd.to_numeric(df.get("capx", np.nan), errors="coerce")
df["sp_cfo_to_debt"] = safe_divide(oancf, df["total_debt"])
df["focf"] = oancf - capx
df["sp_focf_to_debt"] = safe_divide(df["focf"], df["total_debt"])

# --- CHANGED (minimal): make outflows robust to sign conventions ---
dv = pd.to_numeric(df.get("dv", 0.0), errors="coerce").fillna(0.0).abs()
prstkc = pd.to_numeric(df.get("prstkc", 0.0), errors="coerce").fillna(0.0).abs()
df["dcf"] = df["focf"] - dv - prstkc
df["sp_dcf_to_debt"] = safe_divide(df["dcf"], df["total_debt"])
# ------------------------------------------------------------------

# Log transforms (log1p handles 0 nicely). Negative → NaN.
for c in ["at", "mkvalt"]:
    if c in df.columns:
        s = pd.to_numeric(df[c], errors="coerce")
        df[f"log_{c}"] = np.where(s >= 0, np.log1p(s), np.nan)

# Interest coverage: EBITDA / |interest expense|
# Interest coverage can explode when interest expense is near zero.
# Stabilize by flooring the denominator, capping extreme values, and log-transforming.
INT_FLOOR = 1.0  # minimum |interest expense| to avoid blow-ups
IC_CAP = 100.0  # cap extreme coverage magnitudes before log transform
denom_ic = np.maximum(xint.abs(), INT_FLOOR)
df["sp_interest_coverage_raw"] = safe_divide(oibdp, denom_ic)
df["sp_interest_coverage_is_capped"] = (df["sp_interest_coverage_raw"].abs() > IC_CAP).astype("int8")
df["sp_interest_coverage_denom_floored"] = (xint.abs() < INT_FLOOR).astype("int8")
df["sp_interest_coverage"] = np.sign(df["sp_interest_coverage_raw"]) * np.log1p(
    np.minimum(df["sp_interest_coverage_raw"].abs(), IC_CAP)
)

# Distress proxy: 'highly leveraged' condition using multiple credit-ratio thresholds
# NOTE (important): avoid defaulting missing label components to non-distress.
# Pandas BooleanDtype preserves <NA>, so missingness becomes an *unknown* label rather than 0.

td_s  = pd.to_numeric(df["total_debt"], errors="coerce")
cap_s = pd.to_numeric(df["total_capital_sp"], errors="coerce")
eb_s  = pd.to_numeric(oibdp, errors="coerce")
ffo_s = pd.to_numeric(df["ffo_proxy"], errors="coerce")

# Ratios (keep as Series aligned to df.index)
ffo_to_debt_pct = pd.Series(100.0 * safe_divide(ffo_s, td_s), index=df.index, dtype="float64")

debt_to_capital_pct = pd.Series(100.0 * safe_divide(td_s, cap_s), index=df.index, dtype="float64")
# cap <= 0 => extreme leverage (so hl_cap can still trigger)
debt_to_capital_pct = debt_to_capital_pct.mask(cap_s.notna() & (cap_s <= 0), np.inf)

debt_to_ebitda = pd.Series(safe_divide(td_s, eb_s), index=df.index, dtype="float64")
# EBITDA <= 0 => extreme leverage (so hl_deb can still trigger)
debt_to_ebitda = debt_to_ebitda.mask(eb_s.notna() & (eb_s <= 0), np.inf)

def _cmp_with_na(x: pd.Series, op: str, thr: float) -> pd.Series:
    """Comparison that returns <NA> when x is missing (prevents NaN -> False label drift)."""
    if op == "<":
        arr = np.where(x.isna(), pd.NA, x < thr)
    elif op == ">":
        arr = np.where(x.isna(), pd.NA, x > thr)
    else:
        raise ValueError(f"Unsupported op: {op}")
    return pd.Series(pd.array(arr, dtype="boolean"), index=x.index)

# td > 0 should also be <NA> when total debt is missing
td_pos = pd.Series(pd.array(np.where(td_s.isna(), pd.NA, td_s > 0), dtype="boolean"), index=df.index)

# Three “highly leveraged” conditions (S&P table), now missingness-aware
hl_ffo = td_pos & _cmp_with_na(ffo_to_debt_pct, "<", 15.0)      # FFO/total debt < 15%
hl_cap = td_pos & _cmp_with_na(debt_to_capital_pct, ">", 55.0)  # TD/total capital > 55% (or cap<=0 => inf)
hl_deb = td_pos & _cmp_with_na(debt_to_ebitda, ">", 4.5)        # TD/EBITDA > 4.5 (or EBITDA<=0 => inf)

is_highly_leveraged = hl_ffo & hl_cap & hl_deb  # BooleanDtype (can be <NA>)

# Equity strictly negative and not missing (kept as a separate, deterministic distress trigger)
is_equity_negative = (seq.notna() & (seq < 0))
eq_bool = pd.Series(pd.array(is_equity_negative.to_numpy(dtype=bool), dtype="boolean"), index=df.index)

# Final distress rule:
#   - 1 if evidence exists (high leverage or neg equity)
#   - <NA> if high leverage is <NA> and no neg-equity trigger (unknown rather than 0)
distress_bool = is_highly_leveraged | eq_bool
df["distress_dummy"] = distress_bool.astype("Int8")  # nullable integer to preserve <NA>

# Diagnostics (useful for auditability / reporting)
df["distress_label_incomplete"] = df["distress_dummy"].isna().astype("int8")
n_incomplete = int(df["distress_label_incomplete"].sum())
print(
    f"Distress proxy: incomplete label components -> distress_dummy=<NA> for {n_incomplete:,} rows "
    f"({(n_incomplete / max(len(df), 1)):.1%})."
)
# ------------------------------------------------------------------------------

# Target: next year's distress (within firm)
df["target_next_year_distress"] = (
    df.groupby(firm_col)["distress_dummy"].shift(-1)
)

# Indicator for whether the next-year label is observed (attrition / survivorship considerations)
df["has_next_year_obs"] = df["target_next_year_distress"].notna().astype("int8")
n_total = len(df)
n_missing_next = int((df["has_next_year_obs"] == 0).sum())
print(
    f"Next-year label availability: {n_total - n_missing_next:,}/{n_total:,} observed "
    f"({(1 - n_missing_next / max(n_total, 1)):.1%}); missing next-year={n_missing_next:,}."
)

# ---- Attrition diagnostics (next-year label missingness) ----
# Missing next-year observations are rarely random in corporate panels (delistings, M&A, coverage breaks).
# We report simple, publication-friendly diagnostics for selection risk.

try:
    _tmp = df[[firm_col, "fyear", "has_next_year_obs", "distress_dummy", "at"]].copy()
    _tmp["at"] = pd.to_numeric(_tmp["at"], errors="coerce")
    # Size deciles within year to avoid secular size drift
    _tmp["size_decile"] = _tmp.groupby("fyear")["at"].transform(
        lambda s: pd.qcut(s, 10, labels=False, duplicates="drop") + 1
        if s.notna().sum() >= 20 else pd.Series([np.nan] * len(s), index=s.index)
    )
    # 1) By year
    year_attr = (_tmp.groupby("fyear")["has_next_year_obs"]
                 .agg(["count", "mean"])
                 .rename(columns={"count": "n", "mean": "share_observed"}))
    year_attr["attrition_rate"] = 1.0 - year_attr["share_observed"]
    print("\nAttrition by year (target observability):")
    display(year_attr.reset_index().sort_values("fyear"))

    # 2) By current distress state
    state_attr = (_tmp.groupby("distress_dummy")["has_next_year_obs"]
                  .agg(["count", "mean"])
                  .rename(columns={"count": "n", "mean": "share_observed"}))
    state_attr["attrition_rate"] = 1.0 - state_attr["share_observed"]
    print("\nAttrition by current distress state (t):")
    display(state_attr.reset_index())

    # 3) By size decile (pooled)
    if "size_decile" in _tmp.columns:
        size_attr = (_tmp.groupby("size_decile")["has_next_year_obs"]
                     .agg(["count", "mean"])
                     .rename(columns={"count": "n", "mean": "share_observed"}))
        size_attr["attrition_rate"] = 1.0 - size_attr["share_observed"]
        print("\nAttrition by size decile (within-year deciles, pooled):")
        display(size_attr.reset_index().sort_values("size_decile"))
except Exception as e:
    print(f"Attrition diagnostics skipped due to error: {e}")


# Modeling sample: restrict to observed next-year labels (do NOT overwrite df)
df_model = df[df["has_next_year_obs"] == 1].copy().reset_index(drop=True)
df_model["target_next_year_distress"] = df_model["target_next_year_distress"].astype("int8")



In [ ]:
# =============================================================================
# 4. Distinct moments / event indicators (interpretable "drivers & levers")
# Purpose:
#   - Translate continuous accounting ratios into discrete, management-relevant "events"
#   - Use TRAIN pool only for any distribution-based thresholds (no look-ahead)
# =============================================================================

df_events = df.copy()


# ----------------------------
# Helpers
# ----------------------------
def _as_series(x, index) -> pd.Series:
    """Ensure x is a pandas Series aligned to index (handles scalars, arrays, Series)."""
    if isinstance(x, pd.Series):
        return x.reindex(index)
    if isinstance(x, (pd.Index, list, tuple, np.ndarray)):
        return pd.Series(x, index=index, dtype="float64")
    return pd.Series([x] * len(index), index=index, dtype="float64")


def lag(series: pd.Series, k: int = 1) -> pd.Series:
    """Firm-level lag (t-k) using firm_id."""
    s = _as_series(series, df_events.index)
    return s.groupby(df_events["firm_id"]).shift(k)


def ratio(a, b) -> pd.Series:
    """Safe ratio a/b (handles zeros, inf, NaN)."""
    a_s = _as_series(a, df_events.index)
    b_s = _as_series(b, df_events.index)
    return pd.Series(safe_divide(a_s, b_s), index=df_events.index, dtype="float64")


# Define training mask for distribution-aware thresholds (exclude val/test)
pool_mask = (df_events["label_year"] <= TRAIN_CUTOFF_LABEL_YEAR)
train_mask = pool_mask.copy()
if "val_years" in globals() and len(val_years) > 0:
    train_mask = train_mask & (~df_events["label_year"].isin(val_years))

# =============================================================================
# 4.1 Dividend "moments"
# =============================================================================
# Distress literature treats dividend cuts as a signal of cash-flow stress / financing constraints.
# Given the mass at zero (many firms pay no dividends), define events conditional on prior payment.

div = pd.to_numeric(
    _as_series(df_events["dv"] if "dv" in df_events.columns else 0.0, df_events.index),
    errors="coerce",
).fillna(0.0)
div_l1 = lag(div, 1).fillna(0.0)

# Distribution-aware cut threshold among dividend payers (TRAIN pool only)
payer_mask_train = train_mask & (div_l1 > 0)
pct_change = (div - div_l1) / div_l1.replace(0, np.nan)
df_events["div_pct_change"] = pct_change

cut_q = pct_change.loc[payer_mask_train].quantile(0.10)  # 10th percentile among payers
cut_threshold = float(cut_q) if pd.notna(cut_q) else -0.25
cut_threshold = max(min(cut_threshold, -0.10), -0.50)  # clamp to reasonable range

df_events["evt_div_cut"] = ((div_l1 > 0) & (pct_change <= cut_threshold)).astype("int8")
df_events["evt_div_suspend"] = ((div_l1 > 0) & (div <= 0)).astype("int8")
df_events["evt_div_initiate"] = ((div_l1 <= 0) & (div > 0)).astype("int8")

# =============================================================================
# 4.2 / liquidity / profitability "moments"
# =============================================================================

# --- Liquidity squeeze ---
act = pd.to_numeric(_as_series(df_events.get("act", np.nan), df_events.index), errors="coerce")
lct = pd.to_numeric(_as_series(df_events.get("lct", np.nan), df_events.index), errors="coerce")
invt = pd.to_numeric(_as_series(df_events.get("invt", 0.0), df_events.index), errors="coerce").fillna(0.0)

cr = ratio(act, lct)
qr = ratio(act - invt, lct)

df_events["evt_liquidity_squeeze"] = (cr.notna() & (cr < 1.0)).astype("int8")
df_events["evt_quick_squeeze"] = (qr.notna() & (qr < 0.8)).astype("int8")

# --- EBITDA and CFO stress ---
oibdp = pd.to_numeric(_as_series(df_events.get("oibdp", np.nan), df_events.index), errors="coerce")
oibdp_l1 = lag(oibdp, 1)
# Define "drop" as a multiplicative deterioration only where that ratio is meaningful.
doibdp = pd.Series(np.nan, index=df_events.index, dtype="float64")
m_eb = (
    oibdp.notna()
    & oibdp_l1.notna()
    & np.isfinite(oibdp)
    & np.isfinite(oibdp_l1)
    & (oibdp_l1 > 0)
    & (oibdp >= 0)
)
doibdp.loc[m_eb] = (oibdp.loc[m_eb] / oibdp_l1.loc[m_eb]).astype("float64")

df_events["doibdp"] = doibdp
doibdp_cal = doibdp.loc[train_mask].replace([np.inf, -np.inf], np.nan).dropna()
doibdp_q = doibdp_cal.quantile(0.05)
doibdp_thr = float(doibdp_q) if pd.notna(doibdp_q) else 0.5
doibdp_thr = float(np.clip(doibdp_thr, 0.10, 0.90))  # guardrails

df_events["evt_ebitda_drop"] = (m_eb & (doibdp < doibdp_thr)).astype("int8")
df_events["evt_ebitda_neg"] = (oibdp.notna() & (oibdp < 0)).astype("int8")

oancf = pd.to_numeric(_as_series(df_events.get("oancf", np.nan), df_events.index), errors="coerce")
oancf_l1 = lag(oancf, 1)
# Define "drop" as a multiplicative deterioration only where that ratio is meaningful.
dcfo = pd.Series(np.nan, index=df_events.index, dtype="float64")
m_cfo = (
    oancf.notna()
    & oancf_l1.notna()
    & np.isfinite(oancf)
    & np.isfinite(oancf_l1)
    & (oancf_l1 > 0)
    & (oancf >= 0)
)
dcfo.loc[m_cfo] = (oancf.loc[m_cfo] / oancf_l1.loc[m_cfo]).astype("float64")

df_events["dcfo"] = dcfo
dcfo_cal = dcfo.loc[train_mask].replace([np.inf, -np.inf], np.nan).dropna()
cfo_drop_q = dcfo_cal.quantile(0.05)
cfo_drop_thr = float(cfo_drop_q) if pd.notna(cfo_drop_q) else 0.5
cfo_drop_thr = float(np.clip(cfo_drop_thr, 0.10, 0.90))  # guardrails

df_events["evt_cfo_drop"] = (m_cfo & (dcfo < cfo_drop_thr)).astype("int8")
df_events["evt_cfo_neg"] = (oancf.notna() & (oancf < 0)).astype("int8")

# =============================================================================
# 4.3 Sanity summary (TRAIN pool only)
# =============================================================================
event_cols = [c for c in df_events.columns if c.startswith("evt_")]
summary_pool = (
    df_events.loc[train_mask, event_cols]
    .mean()
    .sort_values(ascending=False)
    .to_frame("train_event_rate")
    .reset_index()
    .rename(columns={"index": "event"})
)

print("Dividend cut threshold (TRAIN 10th pct):", round(cut_threshold, 3))
print("EBITDA drop threshold (TRAIN 5th pct):", round(doibdp_thr, 3))
print("CFO drop threshold (TRAIN 5th pct):", round(cfo_drop_thr, 3))

display(summary_pool.head(15))


In [ ]:

# =============================================================================
# Train / Validation / Test Split (out-of-time)
#   - Split by label_year to respect the t → t+1 prediction structure
#   - Keep the last label year(s) inside the training pool as validation
# =============================================================================
# Use the observed-label sample for modeling to avoid silently conditioning on future existence
df_split = df_events[df_events["has_next_year_obs"] == 1].copy().reset_index(drop=True)

train_pool = df_split[df_split["label_year"] <= TRAIN_CUTOFF_LABEL_YEAR].copy()
test = df_split[df_split["label_year"] > TRAIN_CUTOFF_LABEL_YEAR].copy()

years = np.sort(train_pool["label_year"].dropna().unique())
val_years = years[-VAL_YEARS:] if len(years) else np.array([], dtype=int)

val = train_pool[train_pool["label_year"].isin(val_years)].copy()
train = train_pool[~train_pool["label_year"].isin(val_years)].copy()

print(
    "Split:",
    f"train={len(train):,}",
    f"val={len(val):,}",
    f"test={len(test):,}",
    "| val_years:",
    list(val_years),
)

In [ ]:

# =============================================================================
# Modeling-Ready Preprocessing (fit on TRAIN only)
#   - Handle infinities and remaining NaNs
#   - Winsorize continuous features using TRAIN quantile bounds
#   - Standardize continuous features to z-scores using TRAIN stats
#   - Keep binary (event) features as-is
# =============================================================================
continuous_feats = [
    # "sp_debt_to_capital",   # EXCLUDED: part of distress dummy definition
    # "sp_ffo_to_debt",      # EXCLUDED: part of distress dummy definition
    "sp_cfo_to_debt",
    "sp_focf_to_debt",
    "sp_dcf_to_debt",
    # "sp_debt_to_ebitda",    # EXCLUDED: part of distress dummy definition
    # "sp_interest_coverage", # EXCLUDED: coverage ratio proxy (potential circularity)
    "log_at",
    "log_mkvalt",
]
# Ensure they exist in all splits
continuous_feats = [c for c in continuous_feats if c in train.columns and c in val.columns and c in test.columns]

event_feats = []
if "event_cols" in globals():
    # EXCLUDE events derived from the distress-definition variables to avoid endogeneity
    excluded_event_stems = ["evt_cov_breach", "evt_cov_collapse", "evt_lev_spike"]
    event_feats = [
        c for c in event_cols 
        if c in train.columns and c in val.columns and c in test.columns
        and not any(s in c for s in excluded_event_stems)
    ]

# Total features to be used
all_feats = continuous_feats + event_feats

# Replace +/-inf with NaN
for d in (train, val, test):
    d[all_feats] = d[all_feats].replace([np.inf, -np.inf], np.nan)

# Impute NaNs (train-only medians)
fill = train[all_feats].median(numeric_only=True)
for d in (train, val, test):
    d[all_feats] = d[all_feats].fillna(fill)

# Winsorize ONLY continuous features
bounds = {}
for c in continuous_feats:
    s = pd.to_numeric(train[c], errors="coerce")
    bounds[c] = (s.quantile(WINSOR_LOWER_Q), s.quantile(WINSOR_UPPER_Q))

for d in (train, val, test):
    for c, (lo, hi) in bounds.items():
        s = pd.to_numeric(d[c], errors="coerce")
        d[c] = s.clip(lo, hi)

# Standardize (z-scores) ONLY continuous features
scaler = StandardScaler().fit(train[continuous_feats].to_numpy(dtype=float))

# Map to new columns
z_cols_cont = [f"z_{c}" for c in continuous_feats]
train[z_cols_cont] = scaler.transform(train[continuous_feats].to_numpy(dtype=float))
val[z_cols_cont] = scaler.transform(val[continuous_feats].to_numpy(dtype=float))
test[z_cols_cont] = scaler.transform(test[continuous_feats].to_numpy(dtype=float))

# Final model features: scaled continuous + raw binary events
MODEL_FEATS = z_cols_cont + event_feats
# Update z_cols for compatibility with later cells
z_cols = MODEL_FEATS

In [ ]:

# =============================================================================
# Diagnostics & Monitoring Proxies
#   - Correlation screen (TRAIN) for rough signal strength and sanity checks
#   - Expanding-window time folds for temporal stability checks
#   - Dataset overview (rows/firms/years/target rate) + target rate by year
#   - Distribution summaries, collinearity scan, and simple drift proxy (SMD) Train→Test
# =============================================================================
t = "target_next_year_distress"

feats = [c for c in (all_feats if "all_feats" in globals() else z_cols) if c in train.columns and c in test.columns]

corr = (
    train[[t] + feats]
    .corr(numeric_only=True)[t]
    .drop(t)
    .sort_values(key=np.abs, ascending=False)
)
print("Correlation with target:")
print(corr)


# Multicollinearity: Variance Inflation Factor (VIF)
def calculate_vif(df, features):
    vif_data = pd.DataFrame()
    vif_data["feature"] = features
    vif_data["VIF"] = [variance_inflation_factor(df[features].values, i) for i in range(len(features))]
    return vif_data.sort_values("VIF", ascending=False)


vif_df = calculate_vif(train, z_cols)
print("\n=== Multicollinearity Diagnostic (VIF) ===")
print(vif_df)

folds = rolling_year_folds(train_pool, n_splits=N_SPLITS_TIME_CV, min_train_years=3)
for i, (tr_idx, va_idx, tr_years, va_year) in enumerate(folds, 1):
    print(
        f"Fold {i}: train_years={tr_years[0]}..{tr_years[-1]} (n={len(tr_idx)}), "
        f"val_year={va_year} (n={len(va_idx)})"
    )


def _overview(d: pd.DataFrame, name: str) -> None:
    n_rows = len(d)
    n_firms = d["firm_id"].nunique() if "firm_id" in d.columns else np.nan
    n_years = d["fyear"].nunique() if "fyear" in d.columns else np.nan
    target_rate = float(d[t].mean()) if t in d.columns else np.nan

    print(f"\n=== {name} === rows={n_rows:,} | firms={n_firms:,} | years={n_years} | target_rate={target_rate:.4f}")

    if "label_year" in d.columns:
        by_year = d.groupby("label_year")[t].agg(["mean", "count"])
        print("\nTarget by label_year (tail):")
        print(by_year.tail(12))


_overview(train, "TRAIN")
_overview(val, "VAL")
_overview(test, "TEST")

post_miss = pd.DataFrame({"col": raw})
post_miss["train_pct_na"] = [train[c].isna().mean() * 100 if c in train.columns else np.nan for c in raw]
post_miss["val_pct_na"] = [val[c].isna().mean() * 100 if c in val.columns else np.nan for c in raw]
post_miss["test_pct_na"] = [test[c].isna().mean() * 100 if c in test.columns else np.nan for c in raw]
if not post_miss.empty:
    post_miss = post_miss.sort_values("train_pct_na", ascending=False)
    print("\nPost-imputation missingness on raw inputs (pct):")
    print(post_miss.head(50).round(4))


def _dist(d: pd.DataFrame, cols: list[str], name: str) -> pd.DataFrame:
    x = d[cols].replace([np.inf, -np.inf], np.nan)
    q = x.quantile([0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).T
    out = pd.DataFrame(
        {
            "n": x.notna().sum(),
            "mean": x.mean(),
            "std": x.std(ddof=0),
            "min": x.min(),
            "p01": q[0.01],
            "p05": q[0.05],
            "p25": q[0.25],
            "p50": q[0.50],
            "p75": q[0.75],
            "p95": q[0.95],
            "p99": q[0.99],
            "max": x.max(),
            "skew": x.skew(numeric_only=True),
            "kurt": x.kurtosis(numeric_only=True),
        }
    )
    print(f"\nDistribution summary ({name})")
    print(out.round(4).sort_values("skew", key=lambda s: s.abs(), ascending=False))
    return out


_ = _dist(train, feats, "TRAIN | winsorized raw feats")
_ = _dist(train, z_cols, "TRAIN | standardized feats")


def _hi_corr(d: pd.DataFrame, cols: list[str], thr: float = 0.80) -> list[tuple[str, str, float]]:
    cm = d[cols].corr(numeric_only=True)
    pairs = []
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            r = cm.iloc[i, j]
            if np.isfinite(r) and abs(r) >= thr:
                pairs.append((cols[i], cols[j], float(r)))
    return sorted(pairs, key=lambda x: abs(x[2]), reverse=True)


pairs = _hi_corr(train, feats, thr=0.80)
print("\nHigh collinearity pairs among feats (|corr|>=0.80) [top 25]:")
for a, b, r in pairs[:25]:
    print(f"{a} vs {b}: r={r:.3f}")


def _drift_smd(a_df: pd.DataFrame, b_df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    rows = []
    for c in cols:
        a = pd.to_numeric(a_df[c], errors="coerce").replace([np.inf, -np.inf], np.nan)
        b = pd.to_numeric(b_df[c], errors="coerce").replace([np.inf, -np.inf], np.nan)

        ma, mb = float(a.mean()), float(b.mean())
        sa, sb = float(a.std(ddof=0)), float(b.std(ddof=0))
        sp = np.sqrt(0.5 * (sa ** 2 + sb ** 2))
        smd = (mb - ma) / sp if sp > 0 else np.nan

        rows.append((c, ma, mb, smd, abs(smd) if np.isfinite(smd) else np.nan))

        out = pd.DataFrame(rows, columns=["feature", "mean_train", "mean_test", "smd", "abs_smd"])
    return out.sort_values("abs_smd", ascending=False)


drift = _drift_smd(train, test, feats)
print("\nTrain→Test drift (SMD) [top 15]:")
print(drift.head(15).round(4))


def _group_diff(d: pd.DataFrame, cols: list[str]) -> pd.Series:
    g = d.groupby(t)[cols].mean(numeric_only=True)
    if 0 in g.index and 1 in g.index:
        return (g.loc[1] - g.loc[0]).sort_values(key=np.abs, ascending=False)
    return pd.Series(dtype="float64")


diff = _group_diff(train, feats)
if not diff.empty:
    print("\nMean difference (target=1 minus target=0) on TRAIN feats [top 15]:")
    print(diff.head(15).round(4))


In [ ]:
# =============================================================================
# Visual EDA: Feature distributions by distress flag
#   - Quick separation check: do distressed vs non-distressed firms differ in levels?
#   - Uses a horizontal boxplot for comparability across features
# =============================================================================

import matplotlib.pyplot as plt
import seaborn as sns

# Objective: visualize feature distribution differences for distressed vs non-distressed observations.

plot_df = train_pool.copy() if "train_pool" in globals() else df.copy()

flag_col = "distress_dummy" if "distress_dummy" in plot_df.columns else (
    "target_next_year_distress" if "target_next_year_distress" in plot_df.columns else None
)
if flag_col is None:
    raise KeyError("No distress flag found. Expected 'distress_dummy' or 'target_next_year_distress' in the data.")

plot_feats = [c for c in (feats if "feats" in globals() else []) if c in plot_df.columns]
if not plot_feats:
    plot_feats = [c for c in (z_cols if "z_cols" in globals() else []) if c in plot_df.columns]
if not plot_feats:
    raise KeyError(
        "No feature columns found to plot. Expected 'feats' or 'z_cols' to exist and be present in the data.")

tmp = plot_df[[flag_col] + plot_feats].copy()
tmp[flag_col] = pd.to_numeric(tmp[flag_col], errors="coerce").astype("Int64")
tmp = tmp[tmp[flag_col].isin([0, 1])].copy()

long = tmp.melt(id_vars=[flag_col], value_vars=plot_feats, var_name="feature", value_name="value")
long["value"] = pd.to_numeric(long["value"], errors="coerce")
long = long.dropna(subset=["value"])

fig, ax = plt.subplots(figsize=(12, max(4.5, 0.45 * len(plot_feats))))
sns.boxplot(
    data=long,
    x="value",
    y="feature",
    hue=flag_col,
    orient="h",
    showfliers=False,
    ax=ax,
)
ax.set_title(f"Feature distributions by {flag_col} (0=No distress, 1=Distress)")
ax.set_xlabel("Feature value")
ax.set_ylabel("")
ax.legend(title=flag_col, loc="best")
plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# Persistence Benchmark (Economic Sanity Check)
#   - Does the ML model add value over simply assuming distress persists?
#   - We use 'distress_dummy' (current year) to predict 'target_next_year_distress'
# NOTE: distress_dummy can be <NA> if label components are incomplete. Drop those rows here.
# =============================================================================
from sklearn.metrics import f1_score, accuracy_score

for name, d in [("VAL", val), ("TEST", test)]:
    bench = d[[TARGET_COL, "distress_dummy"]].copy()
    bench = bench.dropna(subset=[TARGET_COL, "distress_dummy"])

    if bench.empty:
        print(f"--- Persistence Benchmark ({name}) ---")
        print("No rows available after dropping missing distress_dummy; skipping.")
        continue

    y_true = bench[TARGET_COL].astype(int)
    y_bench = bench["distress_dummy"].astype(int)

    # roc_auc_score requires both classes to be present
    try:
        auc_bench = roc_auc_score(y_true, y_bench)
    except ValueError:
        auc_bench = np.nan

    f1_bench = f1_score(y_true, y_bench)
    acc_bench = accuracy_score(y_true, y_bench)

    print(f"--- Persistence Benchmark ({name}) ---")
    print(f"Rows used: {len(bench):,}/{len(d):,}")
    print(f"AUC: {auc_bench:.4f} | F1: {f1_bench:.4f} | Accuracy: {acc_bench:.4f}")



In [ ]:
# =============================================================================
# 6.1 Setup: Features, target, and train/val/test matrices
# =============================================================================
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    brier_score_loss,
    confusion_matrix,
    classification_report,
    precision_recall_curve,
    roc_curve,
)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

# Prefer standardized feature columns created earlier (fit on TRAIN only)
if "z_cols" in globals():
    MODEL_FEATS = list(z_cols)
else:
    MODEL_FEATS = [f"z_{c}" for c in feats]  # fallback

# Basic sanity checks
for df_name, df_ in [("train", train), ("val", val), ("test", test)]:
    missing_feats = [c for c in MODEL_FEATS if c not in df_.columns]
    if missing_feats:
        raise KeyError(f"{df_name}: missing feature columns: {missing_feats}")

X_train = train[MODEL_FEATS].to_numpy(dtype=float)
y_train = train[TARGET_COL].astype(int).to_numpy()

X_val = val[MODEL_FEATS].to_numpy(dtype=float)
y_val = val[TARGET_COL].astype(int).to_numpy()

X_test = test[MODEL_FEATS].to_numpy(dtype=float)
y_test = test[TARGET_COL].astype(int).to_numpy()

# Defensive: ensure model inputs are finite
def _assert_finite(name, X):
    bad = ~np.isfinite(X)
    if bad.any():
        rows, cols = np.where(bad)
        raise ValueError(f"{name}: found non-finite values at {len(rows)} cells (e.g., row={rows[0]}, col={MODEL_FEATS[cols[0]]}).")

_assert_finite("X_train", X_train)
_assert_finite("X_val", X_val)
_assert_finite("X_test", X_test)

print("Modeling matrix shapes:")
print(f"  X_train: {X_train.shape} | y_train mean={y_train.mean():.4f}")
print(f"  X_val:   {X_val.shape} | y_val mean={y_val.mean():.4f}")
print(f"  X_test:  {X_test.shape} | y_test mean={y_test.mean():.4f}")

In [ ]:
# =============================================================================
# 6.2 Baseline model: Logistic Regression (tuned on VAL)
#   - Interpretable, strong baseline for tabular finance ratios
#   - We tune C on VAL; you can expand the grid later if needed
# =============================================================================
C_GRID = [0.01, 0.1, 1.0, 10.0]

best = {"C": None, "val_auc": -np.inf, "model": None}

for C in C_GRID:
    clf = LogisticRegression(
        C=C,
        solver="lbfgs",
        max_iter=500,
        class_weight=None,  # keep likelihood-consistent probabilities (PD interpretation requires calibration discipline)
        n_jobs=None,
        random_state=42,
    )
    clf.fit(X_train, y_train)
    val_proba = clf.predict_proba(X_val)[:, 1]
    val_auc = roc_auc_score(y_val, val_proba)

    if val_auc > best["val_auc"]:
        best.update({"C": C, "val_auc": val_auc, "model": clf})

print(f"Best LogisticRegression on VAL: C={best['C']} | AUC={best['val_auc']:.4f}")
logit = best["model"]


In [ ]:
# =============================================================================
# 6.2b Statistical Inference & Economic Interpretation (Statsmodels)
#   - sklearn is great for prediction but lacks p-values and inference.
#   - We re-estimate the logit using statsmodels to audit statistical significance.
#   - We also report Marginal Effects to see economic significance.
# =============================================================================
import statsmodels.api as sm

# Add constant for intercept
X_train_sm = sm.add_constant(X_train)
# Panel dependence: use cluster-robust SEs (firm-level) for inference in the training panel
try:
    _groups = train["firm_id"].to_numpy()
    sm_model = sm.Logit(y_train, X_train_sm).fit(
        disp=0,
        cov_type="cluster",
        cov_kwds={"groups": _groups},
    )
    _se_note = "cluster-robust SEs (firm-level)"
except Exception as e:
    # Fallback: still produce estimates, but avoid overstating inference quality
    sm_model = sm.Logit(y_train, X_train_sm).fit(disp=0)
    _se_note = f"non-robust SEs (fallback; clustering failed: {e})"

print(f"\n=== Statsmodels Logistic Regression Summary ({_se_note}) ===")
print(sm_model.summary(xname=["const"] + MODEL_FEATS))

# Marginal Effects at the Mean (MEM)
try:
    mfx = sm_model.get_margeff(at="mean")
    print("\n=== Economic Significance: Marginal Effects at the Mean ===")
    print(mfx.summary())
except Exception as e:
    print(f"\nCould not calculate marginal effects: {e}")

# AIC / BIC / Pseudo-R2
print(f"\nModel Fit:")
print(f"  Pseudo R2 (McFadden): {sm_model.prsquared:.4f}")
print(f"  AIC: {sm_model.aic:.2f}")
print(f"  BIC: {sm_model.bic:.2f}")


In [ ]:
# =============================================================================
# 6.2c Temporal Stability (Walk-Forward Validation)
#   - We assess whether model performance is stable across different time regimes.
#   - Using the expanding-window folds defined in diagnostics.
# =============================================================================
temporal_results = []

FOLD_FEATS = [c for c in feats if c in train_pool.columns]
if not FOLD_FEATS:
    raise KeyError("No usable feature columns found in train_pool for temporal CV. Expected 'feats' columns to exist.")

for i, (tr_idx, va_idx, tr_years, va_year) in enumerate(folds, 1):
    tr_df = train_pool.loc[tr_idx, FOLD_FEATS + [TARGET_COL]].copy()
    va_df = train_pool.loc[va_idx, FOLD_FEATS + [TARGET_COL]].copy()

    # Targets
    y_tr = pd.to_numeric(tr_df[TARGET_COL], errors="coerce").astype(int).to_numpy()
    y_va = pd.to_numeric(va_df[TARGET_COL], errors="coerce").astype(int).to_numpy()

    # Features: fold-local preprocessing (no leakage)
    X_tr_df = tr_df[FOLD_FEATS].replace([np.inf, -np.inf], np.nan).apply(pd.to_numeric, errors="coerce")
    X_va_df = va_df[FOLD_FEATS].replace([np.inf, -np.inf], np.nan).apply(pd.to_numeric, errors="coerce")

    # Median imputation fit on fold-train only
    fold_fill = X_tr_df.median(numeric_only=True)
    X_tr_df = X_tr_df.fillna(fold_fill)
    X_va_df = X_va_df.fillna(fold_fill)

    # Winsorize using fold-train only
    fold_bounds = {}
    for c in FOLD_FEATS:
        s = pd.to_numeric(X_tr_df[c], errors="coerce")
        fold_bounds[c] = (s.quantile(WINSOR_LOWER_Q), s.quantile(WINSOR_UPPER_Q))

    for c, (lo, hi) in fold_bounds.items():
        X_tr_df[c] = pd.to_numeric(X_tr_df[c], errors="coerce").clip(lo, hi)
        X_va_df[c] = pd.to_numeric(X_va_df[c], errors="coerce").clip(lo, hi)

    # Standardize fit on fold-train only
    fold_scaler = StandardScaler().fit(X_tr_df.to_numpy(dtype=float))
    X_tr = fold_scaler.transform(X_tr_df.to_numpy(dtype=float))
    X_va = fold_scaler.transform(X_va_df.to_numpy(dtype=float))

    # Fit model (using best C from tuning)
    fold_clf = LogisticRegression(C=best["C"], solver="lbfgs", max_iter=500, class_weight="balanced", random_state=42)
    fold_clf.fit(X_tr, y_tr)

    # Evaluate
    probs = fold_clf.predict_proba(X_va)[:, 1]
    auc = roc_auc_score(y_va, probs)
    ap = average_precision_score(y_va, probs)

    temporal_results.append({
        "Fold": i,
        "Val_Year": va_year,
        "Train_End": tr_years[-1],
        "AUC": auc,
        "PR-AUC": ap
    })

temporal_df = pd.DataFrame(temporal_results)
print("\n=== Temporal Stability: Walk-Forward Results ===")
print(temporal_df.round(4))
print(f"\nAverage AUC across folds: {temporal_df['AUC'].mean():.4f}")


In [ ]:
# =============================================================================
# 6.3 Evaluation: VAL and TEST (AUC, PR-AUC, Brier) + thresholding
# =============================================================================
def evaluate_split(name, y_true, y_proba, threshold=0.5):
    auc = roc_auc_score(y_true, y_proba)
    ap = average_precision_score(y_true, y_proba)
    brier = brier_score_loss(y_true, y_proba)

    y_pred = (y_proba >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)

    print(f"\n=== {name} ===")
    print(f"AUC (ROC): {auc:.4f}")
    print(f"Average Precision (PR-AUC): {ap:.4f}")
    print(f"Brier score (calibration): {brier:.4f}")
    print(f"Threshold: {threshold:.3f}")
    print("Confusion matrix [ [TN FP] [FN TP] ]:")
    print(cm)
    print("\nClassification report:")
    print(classification_report(y_true, y_pred, digits=4))
    
    # Explicitly pull out Sensitivity and Specificity
    # TN FP
    # FN TP
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"Sensitivity (Recall of Distress): {sensitivity:.4f}")
    print(f"Specificity (Recall of Non-Distress): {specificity:.4f}")
    
    return {"auc": auc, "ap": ap, "brier": brier, "cm": cm, "sens": sensitivity, "spec": specificity}

val_proba = logit.predict_proba(X_val)[:, 1]
test_proba = logit.predict_proba(X_test)[:, 1]

# Choose an operational threshold using VAL under an explicit screening objective.
# Finance deployment typically follows either:
#   (i) capacity constraints (review top X% highest-risk firms), and/or
#   (ii) asymmetric costs (missed distress >> false alarms).
#
# We implement both and report results using the cost-minimizing threshold as the default.

# Capacity rule: flag the top CAP_RATE proportion by predicted risk on VAL
CAP_RATE = 0.10  # adjust to your review bandwidth (e.g., 5%–20%)
cap_thr = float(np.nanquantile(val_proba, 1.0 - CAP_RATE))

# Cost rule: minimize expected cost on VAL over a dense grid of thresholds
# Define relative costs. A principled default is COST_FN proportional to class imbalance,
# reflecting that positives are rarer/more consequential; COST_FP normalized to 1.
pos = float(np.sum(y_val == 1))
neg = float(np.sum(y_val == 0))
imbalance_ratio = (neg / max(pos, 1.0))
COST_FP = 1.0
COST_FN = float(imbalance_ratio)

def expected_cost(y_true, proba, thr, cost_fn, cost_fp):
    y_pred = (proba >= thr).astype(int)
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    return (cost_fp * fp + cost_fn * fn) / max(len(y_true), 1)

thr_grid = np.linspace(0.01, 0.99, 199)
costs = [expected_cost(y_val, val_proba, t, COST_FN, COST_FP) for t in thr_grid]
cost_thr = float(thr_grid[int(np.argmin(costs))])

print(f"VAL base rate: {pos / max(pos + neg, 1.0):.3f} | CAP_RATE={CAP_RATE:.2%} -> cap_thr={cap_thr:.3f} | COST_FN={COST_FN:.2f}, COST_FP={COST_FP:.2f} -> cost_thr={cost_thr:.3f}")

# Report both; use cost_thr as default operating point
_ = evaluate_split("VAL (threshold=min expected cost)", y_val, val_proba, threshold=cost_thr)
_ = evaluate_split("TEST (same threshold)", y_test, test_proba, threshold=cost_thr)

_ = evaluate_split("VAL (capacity threshold: top X% risk)", y_val, val_proba, threshold=cap_thr)
_ = evaluate_split("TEST (capacity threshold: top X% risk)", y_test, test_proba, threshold=cap_thr)


In [ ]:
# =============================================================================
# 6.4 Diagnostic plots: ROC and Precision-Recall (VAL vs TEST)
# =============================================================================
def plot_roc(y_true, y_proba, title):
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    plt.figure()
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.show()

def plot_pr(y_true, y_proba, title):
    p, r, _ = precision_recall_curve(y_true, y_proba)
    plt.figure()
    plt.plot(r, p)
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(title)
    plt.show()

plot_roc(y_val, val_proba, "ROC Curve (VAL) - Logistic Regression")
plot_roc(y_test, test_proba, "ROC Curve (TEST) - Logistic Regression")

plot_pr(y_val, val_proba, "Precision-Recall Curve (VAL) - Logistic Regression")
plot_pr(y_test, test_proba, "Precision-Recall Curve (TEST) - Logistic Regression")


In [ ]:
# =============================================================================
# 6.5 Interpretability: coefficients as (approx.) log-odds contributions
# =============================================================================
coef = pd.Series(logit.coef_.ravel(), index=MODEL_FEATS).sort_values(ascending=False)

summary = pd.DataFrame({
    "feature": coef.index,
    "coef_log_odds": coef.values,
    "odds_ratio": np.exp(coef.values),  # change from 0->1 for binary, per 1SD for scaled continuous
}).sort_values("coef_log_odds", ascending=False)

print("Top positive (higher distress risk):")
display(summary.head(10))

print("Top negative (lower distress risk):")
display(summary.tail(10).iloc[::-1])


In [ ]:
### 6.6  Cost-sensitive boosted trees (non-linear) + calibrated PDs

import numpy as np
import pandas as pd
import xgboost as xgb

from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    confusion_matrix
)
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression
from sklearn.tree import DecisionTreeRegressor, export_text


# =============================================================================
# 0) HELPER UTILITIES
# =============================================================================
def _safe_feature_names(n_features: int):
    cand = globals().get("FEATURE_COLS", globals().get("MODEL_FEATS", None))
    if isinstance(cand, (list, tuple)) and len(cand) == n_features:
        return list(cand)
    return [f"f{i}" for i in range(n_features)]


def gmean_tpr_tnr(y_true, proba, thr):
    y_hat = (proba >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat, labels=[0, 1]).ravel()
    tpr = tp / max(tp + fn, 1)
    tnr = tn / max(tn + fp, 1)
    gmean = float(np.sqrt(tpr * tnr))
    return gmean, tpr, tnr, np.array([[tn, fp], [fn, tp]])


def expected_cost(y_true, proba, thr, cost_fn: float, cost_fp: float):
    y_hat = (proba >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat, labels=[0, 1]).ravel()
    return float(cost_fn * fn + cost_fp * fp), np.array([[tn, fp], [fn, tp]])


def pd_decile_table(y_true, proba, n_bins=10):
    df = pd.DataFrame({"y": np.asarray(y_true).astype(int), "p": np.asarray(proba).astype(float)})
    # qcut can fail with many ties; use rank to stabilize
    r = df["p"].rank(method="first")
    df["bin"] = pd.qcut(r, q=n_bins, labels=False) + 1  # 1..n_bins (1=lowest risk)
    out = (
        df.groupby("bin", as_index=False)
        .agg(n=("y", "size"), realized_rate=("y", "mean"), avg_pd=("p", "mean"),
             min_pd=("p", "min"), max_pd=("p", "max"))
        .sort_values("bin")
    )
    out["lift_vs_base"] = out["realized_rate"] / max(df["y"].mean(), 1e-12)
    return out


# =============================================================================
# 1) SPLIT VALIDATION INTO:
#    - early-stopping set (VAL_ES)
#    - calibration + threshold selection set (VAL_CAL)
#    This prevents using the same data for early stopping AND calibration/thresholding.
# =============================================================================
CAL_SIZE = 0.50  # half of VAL held for calibration + threshold selection

try:
    X_val_es, X_val_cal, y_val_es, y_val_cal = train_test_split(
        X_val, y_val,
        test_size=CAL_SIZE,
        random_state=42,
        stratify=y_val
    )
except Exception:
    # fallback if stratify fails (e.g., too few positives)
    X_val_es, X_val_cal, y_val_es, y_val_cal = train_test_split(
        X_val, y_val,
        test_size=CAL_SIZE,
        random_state=42
    )

# =============================================================================
# 2) COST-SENSITIVE WEIGHTS (weighted cross-entropy spirit: α_FN > α_FP)
#    We implement via sample_weight to avoid double-counting with scale_pos_weight.
# =============================================================================
pos = int(np.sum(y_train))
neg = int(len(y_train) - pos)
imbalance_ratio = neg / max(pos, 1)

# Choose explicit costs (finance logic: FN typically more costly than FP).
# Default: match imbalance ratio -> roughly "balanced" effective loss.
COST_FP = 1.0
COST_FN = float(imbalance_ratio)

w_train = np.where(np.asarray(y_train).astype(int) == 1, COST_FN, COST_FP)
w_val_es = np.where(np.asarray(y_val_es).astype(int) == 1, COST_FN, COST_FP)

# =============================================================================
# 3) XGBOOST SPECIFICATION (strong tabular baseline; modest regularization)
#    Key change: early stopping explicitly on PR-AUC (aucpr), not logloss.
# =============================================================================
base_params = dict(
    objective="binary:logistic",
    booster="gbtree",
    tree_method="hist",
    n_estimators=5000,
    learning_rate=0.02,
    max_depth=4,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=5.0,
    reg_alpha=0.0,
    gamma=0.0,
    max_delta_step=1,
    random_state=42,
    n_jobs=-1,
    # IMPORTANT: do NOT set scale_pos_weight if using sample_weight for costs
    scale_pos_weight=1.0,
)

# Early stop on PR-AUC explicitly (your learning curve peaked in aucpr before logloss)
try:
    es = xgb.callback.EarlyStopping(
        rounds=200,
        metric_name="aucpr",
        data_name="validation_0",
        maximize=True,
        save_best=True
    )
    xgb_clf = XGBClassifier(**base_params, eval_metric=["aucpr", "auc", "logloss"], callbacks=[es])
    xgb_clf.fit(
        X_train, y_train,
        sample_weight=w_train,
        eval_set=[(X_val_es, y_val_es)],
        sample_weight_eval_set=[w_val_es],
        verbose=200
    )
except TypeError:
    # fallback for older API variants
    xgb_clf = XGBClassifier(**base_params, eval_metric=["aucpr", "auc", "logloss"], early_stopping_rounds=200)
    xgb_clf.fit(
        X_train, y_train,
        sample_weight=w_train,
        eval_set=[(X_val_es, y_val_es)],
        sample_weight_eval_set=[w_val_es],
        verbose=200
    )

# =============================================================================
# 4) CALIBRATION (manual isotonic; avoids sklearn cv='prefit' deprecation path)
#    Calibrate ONLY on VAL_CAL (not used for early stopping).
# =============================================================================
raw_cal = xgb_clf.predict_proba(X_val_cal)[:, 1]
raw_test = xgb_clf.predict_proba(X_test)[:, 1]

iso = IsotonicRegression(out_of_bounds="clip")
iso.fit(raw_cal, np.asarray(y_val_cal).astype(int))

val_cal_proba = iso.predict(raw_cal)
test_proba = iso.predict(raw_test)

print("XGBoost (cost-sensitive + early stop on VAL_ES aucpr + isotonic calib on VAL_CAL):")
print(
    f"  VAL_CAL AUC={roc_auc_score(y_val_cal, val_cal_proba):.4f} | "
    f"PR-AUC={average_precision_score(y_val_cal, val_cal_proba):.4f} | "
    f"Brier={brier_score_loss(y_val_cal, val_cal_proba):.4f}"
)
print(
    f"  TEST    AUC={roc_auc_score(y_test, test_proba):.4f} | "
    f"PR-AUC={average_precision_score(y_test, test_proba):.4f} | "
    f"Brier={brier_score_loss(y_test, test_proba):.4f}"
)

# =============================================================================
# 5) THRESHOLDS: (i) cost-based, (ii) G-mean, (iii) capacity (top X%)
#    Thresholds chosen on VAL_CAL; then reported on TEST.
# =============================================================================
grid = np.linspace(0.01, 0.99, 99)

# (i) Cost-based threshold: minimize expected misclassification cost
best_cost = {"thr": 0.5, "cost": np.inf, "cm": None}
for t in grid:
    c, cm = expected_cost(y_val_cal, val_cal_proba, t, cost_fn=COST_FN, cost_fp=COST_FP)
    if c < best_cost["cost"]:
        best_cost.update({"thr": float(t), "cost": float(c), "cm": cm})

# (ii) G-mean threshold (imbalance-robust diagnostic)
best_g = {"thr": 0.5, "gmean": -1, "tpr": None, "tnr": None, "cm": None}
for t in grid:
    g, tpr, tnr, cm = gmean_tpr_tnr(y_val_cal, val_cal_proba, t)
    if g > best_g["gmean"]:
        best_g.update({"thr": float(t), "gmean": g, "tpr": tpr, "tnr": tnr, "cm": cm})

# (iii) Capacity rule: flag top X% by PD (common in finance screening / surveillance)
TOP_PCT = 0.25
thr_top = float(np.quantile(val_cal_proba, 1 - TOP_PCT))


def report_threshold(name, thr):
    g_val, tpr_val, tnr_val, cm_val = gmean_tpr_tnr(y_val_cal, val_cal_proba, thr)
    g_t, tpr_t, tnr_t, cm_t = gmean_tpr_tnr(y_test, test_proba, thr)

    # precision / flag rate
    tn, fp, fn, tp = cm_t.ravel()
    prec = tp / max(tp + fp, 1)
    flag = (tp + fp) / max((tp + fp + tn + fn), 1)

    print(f"\n{name}: threshold t={thr:.3f}")
    print(f"  VAL_CAL  G-mean={g_val:.3f} | TPR={tpr_val:.3f} | TNR={tnr_val:.3f} | CM:\n{cm_val}")
    print(
        f"  TEST     G-mean={g_t:.3f} | TPR={tpr_t:.3f} | TNR={tnr_t:.3f} | Precision={prec:.3f} | FlagRate={flag:.3f} | CM:\n{cm_t}")


report_threshold("Cost-based (min expected cost; COST_FN vs COST_FP)", best_cost["thr"])
report_threshold("G-mean (diagnostic)", best_g["thr"])
report_threshold(f"Top-{int(TOP_PCT * 100)}% capacity rule", thr_top)

# =============================================================================
# 6) FINANCE-STYLE PD SORTS (deciles): monotonicity check + lift
# =============================================================================
print("\nPD Deciles on VAL_CAL (1=lowest risk, 10=highest risk):")
print(pd_decile_table(y_val_cal, val_cal_proba, n_bins=10).to_string(index=False))

print("\nPD Deciles on TEST (1=lowest risk, 10=highest risk):")
print(pd_decile_table(y_test, test_proba, n_bins=10).to_string(index=False))

# =============================================================================
# 7) INTERPRETABILITY: “single-tree approximation” of the calibrated PD surface
#    (practical analogue to merging rules into an interpretable tree)
# =============================================================================
feature_names = _safe_feature_names(X_train.shape[1])

sur = DecisionTreeRegressor(
    max_depth=4,
    min_samples_leaf=max(50, int(0.01 * len(y_train))),
    random_state=42
)

# Fit surrogate on TRAIN predictions (do not use TEST); calibrated PD mapping included
raw_train = xgb_clf.predict_proba(X_train)[:, 1]
train_pd = iso.predict(raw_train)
sur.fit(X_train, train_pd)

print("\nSurrogate tree (depth<=4) approximating calibrated PDs:")
print(export_text(sur, feature_names=feature_names))

# =============================================================================
# 8) FEATURE IMPORTANCE: prefer SHAP; fallback to gain if SHAP unavailable
# =============================================================================
try:
    import shap  # optional

    explainer = shap.TreeExplainer(xgb_clf)
    idx = np.random.RandomState(42).choice(len(X_train), size=min(2000, len(X_train)), replace=False)
    shap_values = explainer.shap_values(X_train[idx])

    imp = np.mean(np.abs(shap_values), axis=0)
    top = np.argsort(-imp)[:20]
    print("\nTop-20 features by mean(|SHAP|):")
    for j in top:
        print(f"{feature_names[j]}: {imp[j]:.6f}")
except Exception:
    booster = xgb_clf.get_booster()
    score = booster.get_score(importance_type="gain")  # {"f0": gain, ...}
    imp = np.zeros(X_train.shape[1], dtype=float)
    for k, v in score.items():
        if isinstance(k, str) and k.startswith("f") and k[1:].isdigit():
            j = int(k[1:])
            if 0 <= j < imp.size:
                imp[j] = float(v)

    top = np.argsort(-imp)[:20]
    print("\nTop-20 features by XGBoost gain (fallback; SHAP unavailable):")
    for j in top:
        print(f"{feature_names[j]}: {imp[j]:.6f}")


In [ ]:
# =============================================================================
# 6.7 Persistence dominance: correct benchmarks for (A) Surveillance vs (B) Early warning
#   - Task A (Surveillance/Triage): allow current state as predictor; compare vs persistence
#   - Task B (Early Warning): restrict to distress(t)=0; predict entry into distress(t+1)
# =============================================================================
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss, log_loss
)


# ----------------------------
# Helpers
# ----------------------------
def _prep_df(d: pd.DataFrame, feats: list[str], target: str, require_state: bool = False) -> pd.DataFrame:
    cols = feats + [target]
    if require_state and "distress_dummy" not in cols:
        cols = cols + ["distress_dummy"]
    out = d[cols].copy()

    # Coerce distress_dummy to numeric if present
    if "distress_dummy" in out.columns:
        out["distress_dummy"] = pd.to_numeric(out["distress_dummy"], errors="coerce")

    # Drop missing
    out = out.dropna(subset=[target] + feats)
    if require_state:
        out = out.dropna(subset=["distress_dummy"])

    # Ensure target is int 0/1
    out[target] = out[target].astype(int)
    return out


def _metrics(y_true: np.ndarray, p: np.ndarray) -> dict:
    y_true = np.asarray(y_true, dtype=int)
    p = np.asarray(p, dtype=float)

    # Clip probabilities for stability (also keeps log_loss happy)
    p = np.clip(p, 1e-15, 1 - 1e-15)

    # handle single-class edge cases for AUC/AP
    out = {}
    try:
        out["AUC"] = float(roc_auc_score(y_true, p))
    except ValueError:
        out["AUC"] = np.nan
    try:
        out["AP"] = float(average_precision_score(y_true, p))
    except ValueError:
        out["AP"] = np.nan

    out["Brier"] = float(brier_score_loss(y_true, p))
    out["LogLoss"] = float(log_loss(y_true, p, labels=[0, 1]))
    out["PosRate"] = float(np.mean(y_true)) if len(y_true) else np.nan
    out["N"] = int(len(y_true))
    return out


def _tune_logit(X_tr, y_tr, X_va, y_va, C_grid=(0.01, 0.1, 1.0, 10.0)):
    best = {"C": None, "va_auc": -np.inf, "model": None}
    for C in C_grid:
        clf = LogisticRegression(
            C=C,
            solver="lbfgs",
            max_iter=800,
            class_weight="balanced",
            random_state=42,
        )
        clf.fit(X_tr, y_tr)
        p_va = clf.predict_proba(X_va)[:, 1]
        try:
            auc = roc_auc_score(y_va, p_va)
        except ValueError:
            auc = np.nan
        if np.isfinite(auc) and auc > best["va_auc"]:
            best = {"C": C, "va_auc": auc, "model": clf}
    return best


def _print_transition_matrix(name: str, d: pd.DataFrame, target_col: str):
    tmp = d[["distress_dummy", target_col]].copy()
    tmp["distress_dummy"] = pd.to_numeric(tmp["distress_dummy"], errors="coerce")
    tmp = tmp.dropna(subset=["distress_dummy", target_col])
    if tmp.empty:
        print(f"\n{name}: (transition matrix unavailable — missing distress_dummy/target)")
        return
    tmp["distress_dummy"] = tmp["distress_dummy"].astype(int)
    tmp[target_col] = tmp[target_col].astype(int)

    # Rows: current state (t), Cols: next-year state (t+1)
    mat = pd.crosstab(tmp["distress_dummy"], tmp[target_col], normalize="index")
    counts = pd.crosstab(tmp["distress_dummy"], tmp[target_col])
    print(f"\n{name}: Transition matrix P(state(t+1) | state(t))")
    display(mat)
    print(f"{name}: Counts")
    display(counts)


# ----------------------------
# Preconditions
# ----------------------------
if "TARGET_COL" not in globals():
    TARGET_COL = "target_next_year_distress"
if "MODEL_FEATS" not in globals():
    raise RuntimeError("MODEL_FEATS not found. Run the preprocessing / feature setup cell first.")

# Transition matrix (this frames persistence dominance properly)
_print_transition_matrix("TRAIN", train, TARGET_COL)
_print_transition_matrix("VAL", val, TARGET_COL)
_print_transition_matrix("TEST", test, TARGET_COL)

# ----------------------------
# TASK A: Surveillance / triage
# ----------------------------
# Baseline: persistence (probability = 1 if distressed now else 0)
# Models: (i) features-only, (ii) features + distress_dummy (increment over persistence)

SURV_FEATS = list(MODEL_FEATS) + ["distress_dummy"]

trainA = _prep_df(train, MODEL_FEATS, TARGET_COL, require_state=True)
valA = _prep_df(val, MODEL_FEATS, TARGET_COL, require_state=True)
testA = _prep_df(test, MODEL_FEATS, TARGET_COL, require_state=True)

trainA_s = _prep_df(train, SURV_FEATS, TARGET_COL, require_state=True)
valA_s = _prep_df(val, SURV_FEATS, TARGET_COL, require_state=True)
testA_s = _prep_df(test, SURV_FEATS, TARGET_COL, require_state=True)

# Persistence baseline probs
# IMPORTANT: persistence is a *hard* classifier; using {0,1} as probabilities makes LogLoss explode.
# Use clipped probabilities for scoring-rule comparability (AUC/AP are unaffected).
eps = 1e-3
p_val_persist  = np.clip(valA["distress_dummy"].to_numpy(dtype=float), 0.0, 1.0)
p_test_persist = np.clip(testA["distress_dummy"].to_numpy(dtype=float), 0.0, 1.0)
p_val_persist  = np.clip(eps + (1.0 - 2.0 * eps) * p_val_persist, eps, 1.0 - eps)
p_test_persist = np.clip(eps + (1.0 - 2.0 * eps) * p_test_persist, eps, 1.0 - eps)

# Fit features-only logit (Task A)
Xtr = trainA[MODEL_FEATS].to_numpy(dtype=float);
ytr = trainA[TARGET_COL].to_numpy(dtype=int)
Xva = valA[MODEL_FEATS].to_numpy(dtype=float);
yva = valA[TARGET_COL].to_numpy(dtype=int)
Xte = testA[MODEL_FEATS].to_numpy(dtype=float);
yte = testA[TARGET_COL].to_numpy(dtype=int)

bestA = _tune_logit(Xtr, ytr, Xva, yva)
logitA = bestA["model"]
p_val_A = logitA.predict_proba(Xva)[:, 1]
p_test_A = logitA.predict_proba(Xte)[:, 1]

# Fit state-augmented logit (Task A incremental)
Xtr_s = trainA_s[SURV_FEATS].to_numpy(dtype=float)
Xva_s = valA_s[SURV_FEATS].to_numpy(dtype=float)
Xte_s = testA_s[SURV_FEATS].to_numpy(dtype=float)
ytr_s = trainA_s[TARGET_COL].to_numpy(dtype=int)
yva_s = valA_s[TARGET_COL].to_numpy(dtype=int)
yte_s = testA_s[TARGET_COL].to_numpy(dtype=int)

bestA_s = _tune_logit(Xtr_s, ytr_s, Xva_s, yva_s)
logitA_s = bestA_s["model"]
p_val_A_s = logitA_s.predict_proba(Xva_s)[:, 1]
p_test_A_s = logitA_s.predict_proba(Xte_s)[:, 1]

rows = []
rows.append(("Task A (Surveillance)", "VAL", "Persistence", *_metrics(yva, p_val_persist).values()))
rows.append(("Task A (Surveillance)", "VAL", f"Logit (feats-only) C={bestA['C']}", *_metrics(yva, p_val_A).values()))
rows.append(
    ("Task A (Surveillance)", "VAL", f"Logit (feats+state) C={bestA_s['C']}", *_metrics(yva_s, p_val_A_s).values()))

rows.append(("Task A (Surveillance)", "TEST", "Persistence", *_metrics(yte, p_test_persist).values()))
rows.append(("Task A (Surveillance)", "TEST", f"Logit (feats-only) C={bestA['C']}", *_metrics(yte, p_test_A).values()))
rows.append(
    ("Task A (Surveillance)", "TEST", f"Logit (feats+state) C={bestA_s['C']}", *_metrics(yte_s, p_test_A_s).values()))

# ----------------------------
# TASK B: Early warning (entry into distress)
# ----------------------------
# Restrict to firms not distressed at t; target remains distress(t+1)=1
trainB = _prep_df(train, MODEL_FEATS + ["distress_dummy"], TARGET_COL, require_state=True)
valB = _prep_df(val, MODEL_FEATS + ["distress_dummy"], TARGET_COL, require_state=True)
testB = _prep_df(test, MODEL_FEATS + ["distress_dummy"], TARGET_COL, require_state=True)

trainB = trainB[trainB["distress_dummy"].astype(int) == 0].copy()
valB = valB[valB["distress_dummy"].astype(int) == 0].copy()
testB = testB[testB["distress_dummy"].astype(int) == 0].copy()

# Baselines for Task B
# - Always-0 (prob=0)
# - Constant transition rate from TRAIN (prob = mean(y_train_B))
ytrB = trainB[TARGET_COL].to_numpy(dtype=int)
p_tr_rate = float(np.mean(ytrB)) if len(ytrB) else 0.0


def _eval_taskB(split_name, dB):
    y = dB[TARGET_COL].to_numpy(dtype=int)
    p0 = np.zeros_like(y, dtype=float)
    pconst = np.full_like(y, fill_value=p_tr_rate, dtype=float)

    # Fit features-only logit on Task B
    XtrB = trainB[MODEL_FEATS].to_numpy(dtype=float)
    XvaB = valB[MODEL_FEATS].to_numpy(dtype=float)
    XteB = testB[MODEL_FEATS].to_numpy(dtype=float)

    yvaB = valB[TARGET_COL].to_numpy(dtype=int)
    yteB = testB[TARGET_COL].to_numpy(dtype=int)

    bestB = _tune_logit(XtrB, ytrB, XvaB, yvaB)
    logitB = bestB["model"]

    if split_name == "VAL":
        p = logitB.predict_proba(XvaB)[:, 1]
        return bestB, [
            ("Task B (Early warning)", "VAL", "Always-0", *_metrics(yvaB, p0).values()),
            ("Task B (Early warning)", "VAL", f"Const rate={p_tr_rate:.4f}", *_metrics(yvaB, pconst).values()),
            ("Task B (Early warning)", "VAL", f"Logit (feats-only) C={bestB['C']}", *_metrics(yvaB, p).values()),
        ]
    else:
        p = logitB.predict_proba(XteB)[:, 1]
        return bestB, [
            ("Task B (Early warning)", "TEST", "Always-0", *_metrics(yteB, p0).values()),
            ("Task B (Early warning)", "TEST", f"Const rate={p_tr_rate:.4f}", *_metrics(yteB, pconst).values()),
            ("Task B (Early warning)", "TEST", f"Logit (feats-only) C={bestB['C']}", *_metrics(yteB, p).values()),
        ]


bestB, rowsB_val = _eval_taskB("VAL", valB)
_, rowsB_tst = _eval_taskB("TEST", testB)

rows.extend(rowsB_val)
rows.extend(rowsB_tst)

# ----------------------------
# Present results
# ----------------------------
cols = ["Task", "Split", "Model", "AUC", "AP", "Brier", "LogLoss", "PosRate", "N"]
results = pd.DataFrame(rows, columns=cols)

# make numeric columns numeric (for sorting/formatting)
for c in ["AUC", "AP", "Brier", "LogLoss", "PosRate"]:
    results[c] = pd.to_numeric(results[c], errors="coerce")

display(results)


In [ ]:
# =============================================================================
# 7.1 Baseline distress rate, event lift, and year clustering (distribution-aware)
# =============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Robust references ---
_df = df_split.copy() if "df_split" in globals() else (df_events.copy() if "df_events" in globals() else df.copy())
_target = TARGET_COL if "TARGET_COL" in globals() else (
    "target_next_year_distress" if "target_next_year_distress" in _df.columns else "distress_dummy"
)
if _target not in _df.columns:
    raise KeyError(f"Target not found. Expected TARGET_COL or 'target_next_year_distress'/'distress_dummy'. Got: {_target}")

_evt_cols = [c for c in _df.columns if c.startswith("evt_")]
if not _evt_cols:
    raise ValueError("No event columns found (expected columns starting with 'evt_').")

# Year for reporting (feature-year)
if "fyear" in _df.columns:
    _year = _df["fyear"]
else:
    # label_year = fyear + 1 (in this project); use feature-year = label_year - 1
    _year = _df["label_year"] - 1 if "label_year" in _df.columns else pd.Series(np.nan, index=_df.index)

base_rate = float(pd.to_numeric(_df[_target], errors="coerce").mean())
print(f"Baseline distress rate (overall): {base_rate:.4%}  |  N={len(_df):,}")

def event_lift_table(df_in: pd.DataFrame, target: str, event_cols: list[str]) -> pd.DataFrame:
    y = pd.to_numeric(df_in[target], errors="coerce")
    base = float(y.mean())
    rows = []
    for e in event_cols:
        s = pd.to_numeric(df_in[e], errors="coerce").fillna(0).astype(int)
        n = int(s.sum())
        prev = float(s.mean())
        cond = float(y[s == 1].mean()) if n > 0 else np.nan
        lift = (cond / base) if (pd.notna(cond) and base > 0) else np.nan
        pp = (cond - base) if pd.notna(cond) else np.nan
        rows.append((e, n, prev, cond, lift, pp))
    out = pd.DataFrame(
        rows,
        columns=["event", "n_events", "event_rate", "distress_rate_given_event", "lift_vs_base", "pp_diff_vs_base"],
    )
    return out.sort_values(["lift_vs_base", "n_events"], ascending=[False, False]).reset_index(drop=True)

lift_all = event_lift_table(_df, _target, _evt_cols)
display(lift_all.head(20))

# --- Year clustering ---
year_tbl = (
    _df.assign(feature_year=_year)
    .groupby("feature_year", dropna=True)[[_target] + _evt_cols]
    .mean(numeric_only=True)
    .sort_index()
)

# Plot: distress rate + a few key event rates (auto-select top by lift)
top_events = list(lift_all["event"].head(6).values)
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(year_tbl.index, year_tbl[_target], marker="o")
ax.set_title("Distress rate by feature-year")
ax.set_xlabel("Feature year")
ax.set_ylabel("Rate")
ax.grid(True, alpha=0.3)
plt.show()

fig, ax = plt.subplots(figsize=(10, 5))
for e in top_events:
    if e in year_tbl.columns:
        ax.plot(year_tbl.index, year_tbl[e], marker="o", label=e)
ax.set_title("Top event rates by feature-year (top by lift)")
ax.set_xlabel("Feature year")
ax.set_ylabel("Event rate")
ax.grid(True, alpha=0.3)
ax.legend(loc="best")
plt.show()


In [ ]:
# =============================================================================
# 7.2 Event co-occurrence and transitions (anticipation)
# =============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

_df = df_split.copy() if "df_split" in globals() else df_events.copy()
_target = TARGET_COL if "TARGET_COL" in globals() else "target_next_year_distress"
_evt_cols = [c for c in _df.columns if c.startswith("evt_")]

# Feature-year for ordering
if "fyear" in _df.columns:
    _df["_feature_year"] = _df["fyear"]
elif "label_year" in _df.columns:
    _df["_feature_year"] = _df["label_year"] - 1
else:
    raise KeyError("Need 'fyear' or 'label_year' to compute event transitions.")

if "firm_id" not in _df.columns:
    raise KeyError("Need 'firm_id' to compute firm-level event transitions.")

E = _df[_evt_cols].fillna(0).astype(int)

# --- Co-occurrence: Jaccard similarity (intersection/union) for binaries ---
A = E.to_numpy(dtype=int)
inter = A.T @ A  # intersection counts
marg = A.sum(axis=0)
union = marg.reshape(-1, 1) + marg.reshape(1, -1) - inter
jacc = np.divide(inter, np.maximum(union, 1), dtype=float)

fig, ax = plt.subplots(figsize=(7, 6))
im = ax.imshow(jacc, aspect="auto")
ax.set_title("Event co-occurrence (Jaccard similarity)")
ax.set_xticks(range(len(_evt_cols)))
ax.set_yticks(range(len(_evt_cols)))
ax.set_xticklabels(_evt_cols, rotation=90, fontsize=7)
ax.set_yticklabels(_evt_cols, fontsize=7)
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()

# --- Transitions: activation (0→1) and persistence (1→1) ---
tmp = _df[["firm_id", "_feature_year"] + _evt_cols].sort_values(["firm_id", "_feature_year"]).copy()
for e in _evt_cols:
    tmp[e + "_next"] = tmp.groupby("firm_id")[e].shift(-1)

rows = []
for e in _evt_cols:
    cur = tmp[e].fillna(0).astype(int)
    nxt = tmp[e + "_next"]
    m = nxt.notna()
    cur_m = cur[m]
    nxt_m = nxt[m].fillna(0).astype(int)

    # activation: P(next=1 | current=0)
    denom01 = int((cur_m == 0).sum())
    p01 = float((nxt_m[(cur_m == 0)] == 1).mean()) if denom01 > 0 else np.nan

    # persistence: P(next=1 | current=1)
    denom11 = int((cur_m == 1).sum())
    p11 = float((nxt_m[(cur_m == 1)] == 1).mean()) if denom11 > 0 else np.nan

    rows.append((e, denom01, p01, denom11, p11))

trans_tbl = pd.DataFrame(rows, columns=["event", "n_cur0", "p_activate_01", "n_cur1", "p_persist_11"])
trans_tbl = trans_tbl.sort_values("p_activate_01", ascending=False).reset_index(drop=True)
display(trans_tbl.head(20))


In [ ]:
# =============================================================================
# 7.3 Tree-based explanations: SHAP focus on event binaries + model-implied ΔPD toggles
# =============================================================================
import numpy as np
import pandas as pd

# Preconditions: trained tree model and calibration (from section 6.6)
if "xgb_clf" not in globals():
    raise RuntimeError("xgb_clf not found. Run the tree-based benchmark cell (6.6) first.")
if "iso" not in globals():
    print("Warning: isotonic calibrator 'iso' not found. Proceeding with raw XGB probabilities.")
if "MODEL_FEATS" not in globals():
    raise RuntimeError("MODEL_FEATS not found. Run the preprocessing / feature setup cells first.")
if "test" not in globals():
    raise RuntimeError("test DataFrame not found.")

_evt_feats = [c for c in MODEL_FEATS if c.startswith("evt_")]
if not _evt_feats:
    raise ValueError("No event binaries found in MODEL_FEATS. Expected event features to be included as raw binaries.")

def _predict_pd(df_rows: pd.DataFrame) -> np.ndarray:
    X = df_rows[MODEL_FEATS].to_numpy(dtype=float)
    raw = xgb_clf.predict_proba(X)[:, 1]
    if "iso" in globals():
        return np.asarray(iso.predict(raw), dtype=float)
    return np.asarray(raw, dtype=float)

# Sample for explanation
N = min(3000, len(test))
sample = test.sample(N, random_state=42).copy()
sample_pd = _predict_pd(sample)

# --- 7.3a Observed conditional PD by event (descriptive) ---
rows = []
for e in _evt_feats:
    s = sample[e].fillna(0).astype(int)
    n1 = int(s.sum())
    if n1 == 0 or n1 == len(sample):
        continue
    pd1 = float(sample_pd[s == 1].mean())
    pd0 = float(sample_pd[s == 0].mean())
    rows.append((e, n1, float(s.mean()), pd0, pd1, pd1 - pd0))

obs_pd_tbl = pd.DataFrame(
    rows, columns=["event", "n_event1", "event_rate_in_sample", "mean_PD_if_0", "mean_PD_if_1", "PD_diff_1_minus_0"]
).sort_values("PD_diff_1_minus_0", ascending=False).reset_index(drop=True)

display(obs_pd_tbl.head(20))

# --- 7.3b Model-implied ΔPD toggles (counterfactual within the model; not causal) ---
def toggle_delta_pd(df_rows: pd.DataFrame, event_col: str, new_value: int) -> np.ndarray:
    tmp = df_rows.copy()
    tmp[event_col] = int(new_value)
    return _predict_pd(tmp)

toggle_rows = []
for e in _evt_feats:
    s = sample[e].fillna(0).astype(int)

    # 0 -> 1 toggle among those currently 0
    m0 = (s == 0)
    if m0.any():
        pd_base0 = sample_pd[m0]
        pd_t1 = toggle_delta_pd(sample.loc[m0, MODEL_FEATS], e, 1)
        d01 = float(np.mean(pd_t1 - pd_base0))
    else:
        d01 = np.nan

    # 1 -> 0 toggle among those currently 1
    m1 = (s == 1)
    if m1.any():
        pd_base1 = sample_pd[m1]
        pd_t0 = toggle_delta_pd(sample.loc[m1, MODEL_FEATS], e, 0)
        d10 = float(np.mean(pd_t0 - pd_base1))
    else:
        d10 = np.nan

    toggle_rows.append((e, float(s.mean()), d01, d10))

toggle_tbl = pd.DataFrame(
    toggle_rows,
    columns=["event", "event_rate_in_sample", "avg_ΔPD_if_toggle_0_to_1", "avg_ΔPD_if_toggle_1_to_0"],
).sort_values("avg_ΔPD_if_toggle_0_to_1", ascending=False).reset_index(drop=True)

display(toggle_tbl.head(25))

# --- Optional: SHAP values (can be slow; uses a smaller sample) ---
try:
    import shap

    N_SHAP = min(1500, len(sample))
    shap_sample = sample.iloc[:N_SHAP].copy()

    explainer = shap.TreeExplainer(xgb_clf)
    shap_values = explainer.shap_values(shap_sample[MODEL_FEATS].to_numpy(dtype=float))
    # shap_values is in log-odds space for binary classifiers (by default)
    shap_df = pd.DataFrame(shap_values, columns=MODEL_FEATS)

    # Summarize absolute SHAP contribution for event features
    evt_shap = (
        shap_df[_evt_feats]
        .abs()
        .mean()
        .sort_values(ascending=False)
        .rename("mean_abs_shap_logit")
        .reset_index()
        .rename(columns={"index": "event"})
    )
    display(evt_shap.head(20))

except Exception as e:
    print("SHAP computation skipped or failed:", repr(e))


In [ ]:
# =============================================================================
# 7.4 Accounting-consistent pro forma scenarios (simulation for a single firm-year)
# =============================================================================
import numpy as np
import pandas as pd

if "xgb_clf" not in globals():
    raise RuntimeError("xgb_clf not found. Run the tree-based benchmark cell (6.6) first.")
if "MODEL_FEATS" not in globals():
    raise RuntimeError("MODEL_FEATS not found. Run preprocessing / feature setup cells first.")
if "continuous_feats" not in globals():
    raise RuntimeError("continuous_feats not found. Expected it from preprocessing cell.")
if "event_feats" not in globals():
    # fall back: infer from MODEL_FEATS
    event_feats = [c for c in MODEL_FEATS if c.startswith("evt_")]

# Required objects from preprocessing (winsor bounds + scaler)
if "bounds" not in globals():
    raise RuntimeError("winsorization bounds not found (expected 'bounds' dict from preprocessing cell).")
if "scaler" not in globals():
    raise RuntimeError("scaler not found (expected StandardScaler 'scaler' from preprocessing cell).")

_df = df_split.copy() if "df_split" in globals() else df_events.copy()


# Utility: calibrated PD
def _predict_pd_from_features(df_rows: pd.DataFrame) -> float | np.ndarray:
    proba = xgb_clf.predict_proba(df_rows[MODEL_FEATS])[:, 1]
    return proba


def _recompute_events_one(raw_row: pd.Series, lag_row: pd.Series | None) -> dict:
    """Recompute event flags for a single row using the same logic as the event cell."""
    out = {}

    # Pull thresholds from the earlier event cell if available; otherwise use conservative defaults
    cut_thr = float(globals().get("cut_threshold", -0.25))
    cov_drop_thr = float(globals().get("cov_drop_thr", 0.7))
    dlev_thr = float(globals().get("dlev_thr", 5.0))
    doibdp_thr = globals().get("doibdp_q", np.nan)
    if isinstance(doibdp_thr, (pd.Series, pd.DataFrame)):
        doibdp_thr = float(pd.to_numeric(doibdp_thr, errors="coerce"))
    cfo_drop_thr = float(globals().get("cfo_drop_thr", 0.7))

    def _num(x):
        return pd.to_numeric(x, errors="coerce")

    def _finite(x) -> bool:
        try:
            return np.isfinite(float(x))
        except Exception:
            return False

    def g(name, default=np.nan):
        return _num(raw_row.get(name, default))

    def glag(name, default=np.nan):
        if lag_row is None:
            return _num(default)
        return _num(lag_row.get(name, default))

    # Dividend moments
    dv = g("dv", 0.0)
    dv_l1 = glag("dv", 0.0)
    pct = (float(dv - dv_l1) / float(dv_l1)) if (_finite(dv) and _finite(dv_l1) and float(dv_l1) != 0.0) else np.nan
    out["evt_div_cut"] = int((_finite(dv_l1) and float(dv_l1) > 0) and pd.notna(pct) and (pct <= cut_thr))
    out["evt_div_suspend"] = int((_finite(dv_l1) and float(dv_l1) > 0) and (_finite(dv) and float(dv) <= 0))
    out["evt_div_initiate"] = int((not _finite(dv_l1) or float(dv_l1) <= 0) and (_finite(dv) and float(dv) > 0))

    # Coverage (RAW interest coverage ratio)
    # NOTE: sp_interest_coverage is a signed log1p-transformed quantity (with caps),
    # so thresholding it at 1.0 is economically incorrect. Events are therefore computed on the RAW ratio.

    cov_raw = g("sp_interest_coverage_raw", np.nan)
    cov_raw_l1 = glag("sp_interest_coverage_raw", np.nan)

    # Fallback reconstruction (if raw coverage column is absent in the selected row(s))
    if not _finite(cov_raw):
        oibdp_now = g("oibdp", np.nan)
        xint_now = g("xint", np.nan)
        denom_now = max(abs(float(xint_now)), 1.0) if _finite(xint_now) else np.nan
        cov_raw = (float(oibdp_now) / denom_now) if (_finite(oibdp_now) and np.isfinite(denom_now) and denom_now != 0.0) else np.nan

    if not _finite(cov_raw_l1):
        oibdp_prev = glag("oibdp", np.nan)
        xint_prev = glag("xint", np.nan)
        denom_prev = max(abs(float(xint_prev)), 1.0) if _finite(xint_prev) else np.nan
        cov_raw_l1 = (float(oibdp_prev) / denom_prev) if (_finite(oibdp_prev) and np.isfinite(denom_prev) and denom_prev != 0.0) else np.nan

    out["evt_cov_breach"] = int(_finite(cov_raw) and (float(cov_raw) < 1.0))  # EBITDA/|interest| < 1 (raw)
    cov_ratio = (cov_raw / cov_raw_l1) if (_finite(cov_raw) and _finite(cov_raw_l1) and float(cov_raw_l1) != 0.0) else np.nan
    out["evt_cov_collapse"] = int(pd.notna(cov_ratio) and (cov_ratio < cov_drop_thr))

    # Leverage spike (debt-to-capital) — guard against inf - inf
    lev = g("sp_debt_to_capital", np.nan)
    lev_l1 = glag("sp_debt_to_capital", np.nan)
    dlev = (float(lev) - float(lev_l1)) if (_finite(lev) and _finite(lev_l1)) else np.nan
    out["evt_lev_spike"] = int(pd.notna(dlev) and (dlev >= dlev_thr))

    # Liquidity squeeze (act/lct and quick ratio)
    act = g("act", np.nan)
    lct = g("lct", np.nan)
    invt = g("invt", 0.0)
    cr = (act / lct) if (_finite(act) and _finite(lct) and float(lct) != 0.0) else np.nan
    qr = ((act - invt) / lct) if (_finite(act) and _finite(lct) and float(lct) != 0.0 and _finite(invt)) else np.nan
    out["evt_liquidity_squeeze"] = int(pd.notna(cr) and (cr < 1.0))
    out["evt_quick_squeeze"] = int(pd.notna(qr) and (qr < 0.8))

    # EBITDA and CFO stress
    oibdp = g("oibdp", np.nan)
    oibdp_l1 = glag("oibdp", np.nan)
    doibdp = (oibdp / oibdp_l1) if (_finite(oibdp) and _finite(oibdp_l1) and float(oibdp_l1) != 0.0) else np.nan
    out["evt_ebitda_drop"] = int(pd.notna(doibdp) and (doibdp < float(doibdp_thr) if np.isfinite(doibdp_thr) else doibdp < 0.7))
    out["evt_ebitda_neg"] = int(_finite(oibdp) and float(oibdp) < 0)

    oancf = g("oancf", np.nan)
    oancf_l1 = glag("oancf", np.nan)
    dcfo = (oancf / oancf_l1) if (_finite(oancf) and _finite(oancf_l1) and float(oancf_l1) != 0.0) else np.nan
    out["evt_cfo_drop"] = int(pd.notna(dcfo) and (dcfo < cfo_drop_thr))
    out["evt_cfo_neg"] = int(_finite(oancf) and float(oancf) < 0)

    return out


def _prepare_feature_row(raw_row: pd.Series, lag_row: pd.Series | None) -> pd.DataFrame:
    """Given a raw row, recompute events, winsorize continuous feats, scale to z, and return 1-row DF with MODEL_FEATS."""
    one = raw_row.copy()

    # Recompute events (only overwrite those present)
    ev = _recompute_events_one(one, lag_row)
    for k, v in ev.items():
        if k in _df.columns:
            one[k] = v

    # Winsorize continuous features using TRAIN bounds (stored in `bounds`)
    for c, (lo, hi) in bounds.items():
        if c in one.index:
            v = pd.to_numeric(one[c], errors="coerce")
            v = np.nan if not np.isfinite(v) else float(v)
            if pd.notna(v) and np.isfinite(lo) and np.isfinite(hi):
                one[c] = float(np.clip(v, lo, hi))
            else:
                one[c] = v

    # Build a 1-row df
    one_df = pd.DataFrame([one])

    # Standardize continuous feats to z_ columns
    one_df[continuous_feats] = one_df[continuous_feats].replace([np.inf, -np.inf], np.nan)
    one_df[continuous_feats] = one_df[continuous_feats].fillna(pd.DataFrame([_df[continuous_feats].median(numeric_only=True)]).iloc[0])

    z_cols = [f"z_{c}" for c in continuous_feats]
    one_df[z_cols] = scaler.transform(one_df[continuous_feats].to_numpy(dtype=float))

    # Ensure event features exist
    for e in event_feats:
        if e not in one_df.columns:
            one_df[e] = 0

    # Final feature row
    return one_df


# Pick a representative firm-year for scenario (highest PD in TEST if available)
if "test" in globals() and "test_proba" in globals():
    idx = int(np.argmax(test_proba))
    base_row = test.iloc[idx].copy()
else:
    base_row = _df.sample(1, random_state=42).iloc[0].copy()

# Pull lagged row if available
lag_row = None
if "firm_id" in _df.columns and "fyear" in _df.columns and "firm_id" in base_row.index:
    fid = base_row["firm_id"]
    y = int(base_row["fyear"])
    lag_match = _df[(_df["firm_id"] == fid) & (_df["fyear"] == (y - 1))]
    if len(lag_match) > 0:
        lag_row = lag_match.iloc[0].copy()

# Base PD
base_feat = _prepare_feature_row(base_row, lag_row)
base_pd = float(_predict_pd_from_features(base_feat)[0])

# Scenarios (simple, interpretable pro-forma adjustments)

scenarios = {
    "Base": {},
    "Dividend suspension (dv→0)": {"dv": 0.0},
    # Note: leverage/coverage scenarios intentionally dropped because leverage/coverage channels are excluded
    # from MODEL_FEATS in this publication specification (to avoid circularity with the label definition).
    "Liquidity buffer (CR>=1.2)": {},  # handled below
}

results = []
for name, adj in scenarios.items():
    row_s = base_row.copy()

    # Apply direct adjustments
    for k, v in adj.items():
        if pd.notna(v) and np.isfinite(v):
            row_s[k] = v

    # Liquidity buffer: minimally raise ACT to reach current ratio >= 1.2 (if inputs exist)
    if name == "Liquidity buffer (CR>=1.2)":
        act = pd.to_numeric(row_s.get("act", np.nan), errors="coerce")
        lct = pd.to_numeric(row_s.get("lct", np.nan), errors="coerce")
        if np.isfinite(act) and np.isfinite(lct) and lct > 0:
            target_act = 1.2 * lct
            row_s["act"] = float(max(act, target_act))

    feat = _prepare_feature_row(row_s, lag_row)

    # Audit: ensure the scenario actually moves at least one MODEL_FEATS column.
    def _neq(a, b):
        # NaN-safe inequality
        if pd.isna(a) and pd.isna(b):
            return False
        try:
            return float(a) != float(b)
        except Exception:
            return a != b

    changed = [c for c in MODEL_FEATS if _neq(feat[c].iloc[0], base_feat[c].iloc[0])]
    pd_s = float(_predict_pd_from_features(feat)[0])

    results.append((name, pd_s, pd_s - base_pd, ", ".join(changed) if changed else "(no MODEL_FEATS change)"))

scen_tbl = (
    pd.DataFrame(results, columns=["Scenario", "PD", "ΔPD vs Base", "MODEL_FEATS changed"])
      .sort_values("PD", ascending=False)
)
print(f"Base PD: {base_pd:.4f}")
display(scen_tbl)


In [ ]:
# =============================================================================
# 7.5 Decision curves (net benefit) + cost curves (screening policy)
# =============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Use calibrated tree PDs if available; otherwise fall back to logistic
if "y_test" in globals() and "test_proba" in globals():
    y = np.asarray(y_test).astype(int)
    p = np.asarray(test_proba).astype(float)
    model_name = "XGBoost (calibrated)"
elif "test" in globals() and "pred_test" in globals() and "p_distress_logit" in pred_test.columns:
    # If you loaded predictions table
    y = np.asarray(pred_test[TARGET_COL]).astype(int)
    p = np.asarray(pred_test["p_distress_logit"]).astype(float)
    model_name = "Logit (probability)"
elif "test" in globals() and "TARGET_COL" in globals() and "test_proba" in globals():
    y = np.asarray(test[TARGET_COL]).astype(int)
    p = np.asarray(test_proba).astype(float)
    model_name = "Model"
else:
    raise RuntimeError("No TEST probabilities found. Run model cells first.")

def net_benefit(y_true, proba, thr):
    y_hat = (proba >= thr).astype(int)
    tp = int(((y_hat == 1) & (y_true == 1)).sum())
    fp = int(((y_hat == 1) & (y_true == 0)).sum())
    n = len(y_true)
    w = thr / max(1 - thr, 1e-12)  # harm/benefit trade-off implied by threshold
    return (tp / n) - (fp / n) * w

ths = np.linspace(0.01, 0.60, 60)
nb_model = np.array([net_benefit(y, p, t) for t in ths])

# Treat-all and treat-none baselines for decision curve analysis
prev = y.mean()
nb_all = prev - (1 - prev) * (ths / (1 - ths))
nb_none = np.zeros_like(ths)

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(ths, nb_model, label=model_name)
ax.plot(ths, nb_all, linestyle="--", label="Treat all")
ax.plot(ths, nb_none, linestyle="--", label="Treat none")
ax.set_title("Decision curve (Net Benefit)")
ax.set_xlabel("Threshold probability")
ax.set_ylabel("Net benefit")
ax.grid(True, alpha=0.3)
ax.legend(loc="best")
plt.show()

# Cost curve: expected cost under FP/FN costs
C_FN = float(globals().get("C_FN", 5.0))  # default: FN 5x as costly as FP
C_FP = float(globals().get("C_FP", 1.0))

def expected_cost(y_true, proba, thr, c_fn=C_FN, c_fp=C_FP):
    y_hat = (proba >= thr).astype(int)
    fn = int(((y_hat == 0) & (y_true == 1)).sum())
    fp = int(((y_hat == 1) & (y_true == 0)).sum())
    return (c_fn * fn + c_fp * fp) / max(len(y_true), 1)

costs = np.array([expected_cost(y, p, t) for t in ths])
fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(ths, costs)
ax.set_title(f"Expected cost vs threshold (C_FN={C_FN:g}, C_FP={C_FP:g})")
ax.set_xlabel("Threshold probability")
ax.set_ylabel("Expected cost per observation")
ax.grid(True, alpha=0.3)
plt.show()
